In [ ]:
# Install necessary packages
!git clone https://github.com/ultralytics/yolov5  # Clone YOLOv5 repository
%cd yolov5

!pip install -qr requirements.txt comet_ml  # Install YOLOv5 dependencies
!pip install streamlit opencv-python-headless pyngrok torchvision iopath torch moviepy scikit-learn pillow

!npm install -g localtunnel

# Install SlowFast and PyTorchVideo
!git clone https://github.com/facebookresearch/slowfast.git
%cd slowfast
!python setup.py build develop

!pip uninstall -y pytorchvideo
!git clone https://github.com/facebookresearch/pytorchvideo.git
%cd pytorchvideo
!pip install -e .

# Streamlit app code
app_code = """
import streamlit as st
import os
import shutil
from PIL import Image
import subprocess
import torch
from pytorchvideo.models.hub import slowfast_r50
import torchvision.transforms as transforms
from pytorchvideo.transforms import ShortSideScale, UniformTemporalSubsample
from pytorchvideo.data.encoded_video import EncodedVideo

# Ensure detect.py is present
assert os.path.exists('detect.py'), 'detect.py not found. Ensure it is in the current directory.'

def process_video(video_path):
    # Run detect.py on the uploaded video
    output_dir = 'runs/detect/exp'
    video_output = os.path.join(output_dir, 'uploaded_video.mp4')  # Adjust the filename as necessary

    if os.path.exists(output_dir):
        shutil.rmtree(output_dir)  # Clear previous runs

    command = ['python', 'detect.py', '--weights', 'yolov5s.pt', '--source', video_path, '--classes', '67', '--conf', '0.10', '--save-txt', '--save-conf', '--project', 'runs/detect', '--exist-ok', '--save-crop']
    result = subprocess.run(command, capture_output=True, text=True)

    # Check if the video was processed successfully
    if not os.path.exists(video_output):
        raise Exception("YOLOv5 did not produce the expected output video.")

    return video_output

def display_behavior_predictions(video_path):
    # Load the pre-trained SlowFast model
    model = slowfast_r50(pretrained=True)
    model = model.eval()

    # Load the video using PyTorchVideo's EncodedVideo class
    video = EncodedVideo.from_path(video_path)

    # Select the time range to sample from the video (e.g., from 0 to 15 seconds)
    start_time = 0
    end_time = 15  # In seconds

    # Sample frames uniformly
    video_data = video.get_clip(start_sec=start_time, end_sec=end_time)
    frames = video_data["video"]  # [T, C, H, W] - this should be the shape after this step

    # Define the transforms to apply to each frame
    transform = transforms.Compose([
        UniformTemporalSubsample(32),  # Sample 32 frames uniformly
        ShortSideScale(256),
        transforms.CenterCrop(224),  # Add center crop to get 224x224
        transforms.Lambda(lambda x: x / 255.0),  # Normalize to [0, 1]
    ])

    # Apply the initial transforms
    frames = transform(frames)  # [T, C, H, W] format is preserved

    # Now normalize the RGB channels
    normalize = transforms.Normalize(mean=[0.45, 0.45, 0.45], std=[0.225, 0.225, 0.225])
    frames = frames.permute(1, 0, 2, 3)  # Change to [C, T, H, W]
    frames = normalize(frames)
    frames = frames.permute(1, 0, 2, 3)  # Change back to [T, C, H, W]

    # Prepare the input format for SlowFast
    fast_pathway = frames
    slow_pathway = torch.index_select(
        frames,
        1,
        torch.linspace(0, frames.shape[1] - 1, frames.shape[1] // 4).long(),
    )

    # Combine pathways into a list and add a batch dimension
    inputs = [slow_pathway.unsqueeze(0), fast_pathway.unsqueeze(0)]

    # Run inference
    with torch.no_grad():
        preds = model(inputs)

    # Load Kinetics-400 class labels
    kinetics_labels = []
    with open('/content/slowfast/dataset/class_names/kinetics_classnames.txt', 'r') as f:
        for line in f:
            kinetics_labels.append(line.strip())

    # Print top 5 predictions
    top5_pred = torch.topk(preds, k=5)
    st.write("Top 5 predictions:")
    for i in range(5):
        st.write(f"{i+1}. {kinetics_labels[top5_pred.indices[0][i]]}: {top5_pred.values[0][i]:.3f}")

    return video_path

# Streamlit app
st.title('Object and Behavior Detection in Videos')

uploaded_file = st.file_uploader('Upload a video file', type=['mp4', 'avi', 'mov'])

if uploaded_file is not None:
    video_path = f'uploaded_video.{uploaded_file.type.split("/")[1]}'
    with open(video_path, "wb") as f:
        f.write(uploaded_file.getbuffer())

    st.video(video_path)
    st.write('Running YOLOv5 object detection...')

    try:
        annotated_video_path = process_video(video_path)
        st.write('YOLOv5 detection completed.')
        st.video(annotated_video_path, format='video/mp4', start_time=0)

    except Exception as e:
        st.write(f'Error during YOLOv5 inference: {e}')

    st.write('Running behavior detection...')
    try:
        behavior_video_path = display_behavior_predictions(video_path)
        st.video(behavior_video_path, format='video/mp4', start_time=0)
    except Exception as e:
        st.write(f'Error during behavior inference: {e}')
"""

# Write the Streamlit app code to a file
with open("app.py", "w") as file:
    file.write(app_code)

print("app.py file has been written.")

import subprocess

# Start the Streamlit app in the background
subprocess.Popen(['streamlit', 'run', 'app.py'])

import requests
import time
import subprocess

# Wait a few seconds for the Streamlit app to start
time.sleep(5)

# Start LocalTunnel
localtunnel_process = subprocess.Popen(['lt', '--port', '8501'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

# Capture the LocalTunnel URL
public_url = None
for line in iter(localtunnel_process.stdout.readline, b''):
    decoded_line = line.decode().strip()
    print(decoded_line)
    if 'your url is:' in decoded_line:
        public_url = decoded_line.split(' ')[-1]
        break

if public_url:
    print(f'Streamlit app running at: {public_url}')
    # Make a request to the LocalTunnel URL with custom headers to bypass the password screen
    headers = {
        'User-Agent': 'Mozilla/5.0',
        'bypass-tunnel-reminder': '1'
    }
    response = requests.get(public_url, headers=headers)
    if response.status_code == 200:
        print(f'Access the Streamlit app at: {public_url}')
    else:
        print('Failed to bypass the password screen.')
else:
    print('Failed to create LocalTunnel.')

Cloning into 'yolov5'...
remote: Enumerating objects: 16927, done.
remote: Counting objects: 100% (122/122), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 16927 (delta 62), reused 87 (delta 48), pack-reused 16805 (from 1)
Receiving objects: 100% (16927/16927), 15.68 MiB | 8.29 MiB/s, done.
Resolving deltas: 100% (11600/11600), done.
/content/yolov5
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 687.3/687.3 kB 43.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 106.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 871.6/871.6 kB 57.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 979.1/979.1 kB 43.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 34.6 MB/s e

In [ ]:
!wget -q -O - https://loca.lt/mytunnelpassword

35.186.157.129

In [ ]:
# Install required packages
!pip install streamlit opencv-python-headless pyngrok pytorchvideo torchvision iopath torch fairscale psutil moviepy scikit-learn pillow
!npm install -g localtunnel

# Clone repositories
!git clone https://github.com/ultralytics/yolov5
!git clone https://github.com/facebookresearch/slowfast
!git clone https://github.com/facebookresearch/pytorchvideo

# Install SlowFast
%cd slowfast
!python setup.py build develop
%cd ..

# Install PyTorchVideo
%cd pytorchvideo
!pip install -e .
%cd ..

# Set up PYTHONPATH
import os
os.environ['PYTHONPATH'] += ":/content/slowfast"

# YOLOv5 setup
%cd yolov5
!pip install -qr requirements.txt comet_ml

import torch
import utils
display = utils.notebook_init()

# Train YOLOv5s on COCO128 for 3 epochs (you can comment this out if you don't need to train)
!python train.py --img 640 --batch 16 --epochs 3 --data coco128.yaml --weights yolov5s.pt --cache

# Create Streamlit app
app_code = """
import streamlit as st
import os
import shutil
from PIL import Image
import subprocess
import torch
import torchvision.transforms as transforms
from pytorchvideo.models.hub import slowfast_r50
from pytorchvideo.transforms import ShortSideScale, UniformTemporalSubsample
from pytorchvideo.data.encoded_video import EncodedVideo
import torch.nn.functional as F
import cv2

# Ensure detect.py is present
assert os.path.exists('detect.py'), 'detect.py not found. Ensure it is in the current directory.'

def process_video_yolo(video_path):
    output_dir = 'runs/detect/exp'
    video_output = os.path.join(output_dir, 'uploaded_video.mp4')
    if os.path.exists(output_dir):
        shutil.rmtree(output_dir)  # Clear previous runs
    print(f'Running detect.py on {video_path}')
    command = ['python', 'detect.py', '--weights', 'yolov5s.pt', '--source', video_path, '--classes', '67', '--conf', '0.10', '--save-txt', '--save-conf', '--project', 'runs/detect', '--exist-ok']
    result = subprocess.run(command, capture_output=True, text=True)
    print('detect.py has finished running')
    print(result.stdout)
    print(result.stderr)
    return video_output

def process_video_pytorchvideo(video_path):
    # Load the pre-trained SlowFast model
    model = slowfast_r50(pretrained=True)
    model = model.eval()

    # Load the video using PyTorchVideo's EncodedVideo class
    video = EncodedVideo.from_path(video_path)

    # Select the time range to sample from the video
    start_time = 0
    end_time = min(video.duration, 15)  # Use the shorter of 15 seconds or video duration

    # Sample frames uniformly
    video_data = video.get_clip(start_sec=start_time, end_sec=end_time)
    frames = video_data["video"]

    # Define the transforms to apply to each frame
    transform = transforms.Compose([
        UniformTemporalSubsample(32),
        ShortSideScale(256),
        transforms.CenterCrop(224),
        transforms.Lambda(lambda x: x / 255.0),
    ])

    # Apply the initial transforms
    frames = transform(frames)

    # Normalize the RGB channels
    normalize = transforms.Normalize(mean=[0.45, 0.45, 0.45], std=[0.225, 0.225, 0.225])
    frames = frames.permute(1, 0, 2, 3)
    frames = normalize(frames)
    frames = frames.permute(1, 0, 2, 3)

    # Prepare the input format for SlowFast
    fast_pathway = frames
    slow_pathway = torch.index_select(
        frames,
        1,
        torch.linspace(0, frames.shape[1] - 1, frames.shape[1] // 4).long(),
    )

    # Combine pathways into a list and add a batch dimension
    inputs = [slow_pathway.unsqueeze(0), fast_pathway.unsqueeze(0)]

    # Run inference
    with torch.no_grad():
        preds = model(inputs)

    # Load Kinetics-400 class labels
    kinetics_labels = []
    with open('/content/label_map.txt', 'r') as f:
        for line in f:
            kinetics_labels.append(line.strip())

    # Get top 5 predictions
    top5_pred = torch.topk(preds, k=5)
    top5_actions = []
    for i in range(5):
        action = kinetics_labels[top5_pred.indices[0][i]]
        confidence = top5_pred.values[0][i].item()
        top5_actions.append((action, confidence))

    return top5_actions

# Streamlit app
st.title('YOLOv5 Cell Phone Detection and PyTorchVideo Action Recognition')

uploaded_file = st.file_uploader('Upload a video file', type=['mp4', 'avi', 'mov'])

if uploaded_file is not None:
    video_path = f'uploaded_video.{uploaded_file.type.split("/")[1]}'
    with open(video_path, "wb") as f:
        f.write(uploaded_file.getbuffer())

    st.video(video_path)

    # YOLOv5 Object Detection
    st.subheader('YOLOv5 Cell Phone Detection')
    st.write('Running YOLOv5 inference...')
    try:
        annotated_video_path = process_video_yolo(video_path)
        st.write('YOLOv5 inference completed.')

        if os.path.exists(annotated_video_path):
            st.video(annotated_video_path, format='video/mp4', start_time=0)
            st.write('Cell phones detected in the video.')
        else:
            st.write('No cell phones detected in the video.')
    except Exception as e:
        st.write(f'Error during YOLOv5 inference: {e}')

    # PyTorchVideo Action Recognition
    st.subheader('PyTorchVideo Action Recognition')
    st.write('Running PyTorchVideo inference...')
    try:
        top5_actions = process_video_pytorchvideo(video_path)
        st.write('PyTorchVideo inference completed.')

        st.write('Top 5 predicted actions:')
        for i, (action, confidence) in enumerate(top5_actions, 1):
            st.write(f"{i}. {action}: {confidence:.3f}")
    except Exception as e:
        st.write(f'Error during PyTorchVideo inference: {e}')
"""

with open("app.py", "w") as file:
    file.write(app_code)
print("app.py file has been written.")

# Start the Streamlit app in the background
import subprocess
subprocess.Popen(['streamlit', 'run', 'app.py'])

import requests
import time

# Wait a few seconds for the Streamlit app to start
time.sleep(5)

# Start LocalTunnel
localtunnel_process = subprocess.Popen(['lt', '--port', '8501'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

# Capture the LocalTunnel URL
public_url = None
for line in iter(localtunnel_process.stdout.readline, b''):
    decoded_line = line.decode().strip()
    print(decoded_line)
    if 'your url is:' in decoded_line:
        public_url = decoded_line.split(' ')[-1]
        break

if public_url:
    print(f'Streamlit app running at: {public_url}')
    # Make a request to the LocalTunnel URL with custom headers to bypass the password screen
    headers = {
        'User-Agent': 'Mozilla/5.0',
        'bypass-tunnel-reminder': '1'
    }
    response = requests.get(public_url, headers=headers)
    if response.status_code == 200:
        print(f'Access the Streamlit app at: {public_url}')
    else:
        print('Failed to bypass the password screen.')
else:
    print('Failed to create LocalTunnel.')

# Download the Kinetics-400 label map
!wget -O /content/label_map.txt https://raw.githubusercontent.com/pytorch/vision/main/torchvision/datasets/kinetics.py
# Extract the class names
!sed -n "/'[^']*':/p" /content/label_map.txt | sed "s/[^']*'\([^']*\)':.*/\1/" > /content/label_map.txt

YOLOv5 🚀 v7.0-362-ge8a30cf8 Python-3.10.12 torch-2.4.0+cu121 CUDA:0 (Tesla T4, 15102MiB)


Setup complete ✅ (2 CPUs, 12.7 GB RAM, 33.3/78.2 GB disk)
2024-08-29 03:20:57.208020: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-29 03:20:57.228122: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-29 03:20:57.234060: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
train: weights=yolov5s.pt, cfg=, data=coco128.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=3, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=data/hyps, resume_evolve=None, bucket=, cache=ram, image_weights

In [ ]:
!wget -q -O - https://loca.lt/mytunnelpassword

34.142.246.29

In [ ]:
# Install necessary packages
!pip install streamlit opencv-python-headless pyngrok pytorchvideo torchvision iopath torch git+https://github.com/facebookresearch/fvcore.git git+https://github.com/facebookresearch/fairscale.git
!npm install -g localtunnel

# Clone repositories
!git clone https://github.com/ultralytics/yolov5
!git clone https://github.com/facebookresearch/slowfast
!git clone https://github.com/facebookresearch/pytorchvideo

# Set up YOLOv5
%cd yolov5
!pip install -qr requirements.txt comet_ml
%cd ..

# Set up SlowFast
%cd slowfast
!python setup.py build develop
%cd ..

# Set up PyTorchVideo
%cd pytorchvideo
!pip install -e .
%cd ..

# Add SlowFast to PYTHONPATH
import os
os.environ['PYTHONPATH'] += ":/content/slowfast"

# Create app.py
app_code = """
import streamlit as st
import os
import shutil
from PIL import Image
import subprocess
import torch
from pytorchvideo.models.hub import slowfast_r50
import torchvision.transforms as transforms
from pytorchvideo.transforms import ShortSideScale, UniformTemporalSubsample
from pytorchvideo.data.encoded_video import EncodedVideo
import torch.nn.functional as F
import cv2

# Ensure detect.py is present
assert os.path.exists('yolov5/detect.py'), 'detect.py not found. Ensure it is in the yolov5 directory.'

def process_video_yolov5(video_path):
    # Run detect.py on the uploaded video
    output_dir = 'runs/detect/exp'
    video_output = os.path.join(output_dir, os.path.basename(video_path))
    crops_dir = os.path.join(output_dir, 'crops', 'cell phone')
    if os.path.exists(output_dir):
        shutil.rmtree(output_dir)  # Clear previous runs
    print(f'Running detect.py on {video_path}')
    command = ['python', 'yolov5/detect.py', '--weights', 'yolov5/yolov5s.pt', '--source', video_path, '--classes', '67', '--conf', '0.25', '--save-txt', '--save-conf', '--project', 'runs/detect', '--exist-ok', '--save-crop']
    result = subprocess.run(command, capture_output=True, text=True)
    print('detect.py has finished running')
    print(result.stdout)
    print(result.stderr)

    # Check if cell phones were detected
    cell_phones_detected = os.path.exists(crops_dir) and len(os.listdir(crops_dir)) > 0

    return cell_phones_detected, video_output

def process_video_pytorchvideo(video_path):
    # Load the pre-trained SlowFast model
    model = slowfast_r50(pretrained=True)
    model = model.eval()

    # Load the video using PyTorchVideo's EncodedVideo class
    video = EncodedVideo.from_path(video_path)

    # Select the time range to sample from the video
    start_time = 0
    end_time = min(video.duration, 15)  # Use the shorter of 15 seconds or video duration

    # Sample frames uniformly
    video_data = video.get_clip(start_sec=start_time, end_sec=end_time)
    frames = video_data["video"]

    # Define the transforms to apply to each frame
    transform = transforms.Compose([
        UniformTemporalSubsample(32),
        ShortSideScale(256),
        transforms.CenterCrop(224),
        transforms.Lambda(lambda x: x / 255.0),
    ])

    # Apply the initial transforms
    frames = transform(frames)

    # Normalize the RGB channels
    normalize = transforms.Normalize(mean=[0.45, 0.45, 0.45], std=[0.225, 0.225, 0.225])
    frames = frames.permute(1, 0, 2, 3)
    frames = normalize(frames)
    frames = frames.permute(1, 0, 2, 3)

    # Prepare the input format for SlowFast
    fast_pathway = frames
    slow_pathway = torch.index_select(
        frames,
        1,
        torch.linspace(0, frames.shape[1] - 1, frames.shape[1] // 4).long(),
    )

    # Combine pathways into a list and add a batch dimension
    inputs = [slow_pathway.unsqueeze(0), fast_pathway.unsqueeze(0)]

    # Run inference
    with torch.no_grad():
        preds = model(inputs)

    # Load Kinetics-400 class labels
    kinetics_labels = []
    with open('kinetics_400_labels.txt', 'r') as f:
        for line in f:
            kinetics_labels.append(line.strip())

    # Get top 5 predictions
    top5_pred = torch.topk(preds, k=5)
    top5_actions = []
    for i in range(5):
        action = kinetics_labels[top5_pred.indices[0][i]]
        confidence = top5_pred.values[0][i].item()
        top5_actions.append((action, confidence))

    return top5_actions

# Streamlit app
st.title('YOLOv5 Cell Phone Detection and PyTorchVideo Action Recognition')

uploaded_file = st.file_uploader('Upload a video file', type=['mp4', 'avi', 'mov'])

if uploaded_file is not None:
    video_path = f'uploaded_video.{uploaded_file.type.split("/")[1]}'
    with open(video_path, "wb") as f:
        f.write(uploaded_file.getbuffer())

    st.video(video_path)

    # YOLOv5 Object Detection
    st.write('Running YOLOv5 inference for cell phone detection...')
    try:
        cell_phones_detected, annotated_video_path = process_video_yolov5(video_path)
        st.write('YOLOv5 inference completed.')

        if cell_phones_detected:
            st.write('Cell phones detected in the video.')
            st.video(annotated_video_path)
        else:
            st.write('No cell phones detected in the video.')

    except Exception as e:
        st.write(f'Error during YOLOv5 inference: {e}')

    # PyTorchVideo Action Recognition
    st.write('Running PyTorchVideo inference for action recognition...')
    try:
        top5_actions = process_video_pytorchvideo(video_path)
        st.write('PyTorchVideo inference completed.')

        st.write('Top 5 detected actions:')
        for action, confidence in top5_actions:
            st.write(f"{action}: {confidence:.3f}")

    except Exception as e:
        st.write(f'Error during PyTorchVideo inference: {e}')
"""

with open("app.py", "w") as file:
    file.write(app_code)
print("app.py file has been written.")

# Download Kinetics-400 labels
!wget https://raw.githubusercontent.com/pytorch/vision/main/torchvision/datasets/kinetics.py
!python -c "from kinetics import _KINETICS400_CATEGORIES; print('\n'.join(_KINETICS400_CATEGORIES))" > kinetics_400_labels.txt

# Start the Streamlit app in the background
import subprocess
streamlit_process = subprocess.Popen(['streamlit', 'run', 'app.py'])

# Start LocalTunnel
import time
import requests

time.sleep(5)
localtunnel_process = subprocess.Popen(['lt', '--port', '8501'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

# Capture the LocalTunnel URL
public_url = None
for line in iter(localtunnel_process.stdout.readline, b''):
    decoded_line = line.decode().strip()
    print(decoded_line)
    if 'your url is:' in decoded_line:
        public_url = decoded_line.split(' ')[-1]
        break

if public_url:
    print(f'Streamlit app running at: {public_url}')
    headers = {
        'User-Agent': 'Mozilla/5.0',
        'bypass-tunnel-reminder': '1'
    }
    response = requests.get(public_url, headers=headers)
    if response.status_code == 200:
        print(f'Access the Streamlit app at: {public_url}')
    else:
        print('Failed to bypass the password screen.')
else:
    print('Failed to create LocalTunnel.')

  Cloning https://github.com/facebookresearch/fvcore.git to /tmp/pip-req-build-x0tpskwf
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/fvcore.git /tmp/pip-req-build-x0tpskwf
  Resolved https://github.com/facebookresearch/fvcore.git to commit f3b07ea37daff5b774702708134f7b702e2dc83e
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/facebookresearch/fairscale.git to /tmp/pip-req-build-fb8pjib4
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/fairscale.git /tmp/pip-req-build-fb8pjib4
  Resolved https://github.com/facebookresearch/fairscale.git to commit 5f484b3545f27eddb19d970fbe1d361b9c5f2b07
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.7/132.7 kB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!wget -q -O - https://loca.lt/mytunnelpassword

34.16.223.99

In [ ]:
# Install necessary packages
!pip install streamlit opencv-python-headless pyngrok pytorchvideo torchvision iopath torch
!npm install -g localtunnel

# Clone repositories
!git clone https://github.com/ultralytics/yolov5
!git clone https://github.com/facebookresearch/slowfast
!git clone https://github.com/facebookresearch/pytorchvideo

# Set up YOLOv5
%cd yolov5
!pip install -qr requirements.txt comet_ml
%cd ..

# Set up SlowFast
%cd slowfast
!python setup.py build develop
%cd ..

# Set up PyTorchVideo
%cd pytorchvideo
!pip install -e .
%cd ..

# Add SlowFast to PYTHONPATH
import os
os.environ['PYTHONPATH'] += ":/content/slowfast"

# Download Kinetics-400 labels
!wget -O kinetics_400_labels.txt https://raw.githubusercontent.com/google-deepmind/kinetics-i3d/master/data/label_map.txt

# Create app.py
app_code = """
import streamlit as st
import os
import shutil
import subprocess
import torch
from pytorchvideo.models.hub import slowfast_r50
import torchvision.transforms as transforms
from pytorchvideo.transforms import ShortSideScale, UniformTemporalSubsample
from pytorchvideo.data.encoded_video import EncodedVideo
import torch.nn.functional as F
import cv2
import base64

# Ensure detect.py is present
assert os.path.exists('yolov5/detect.py'), 'detect.py not found. Ensure it is in the yolov5 directory.'

def process_video_yolov5(video_path):
    # Run detect.py on the uploaded video
    output_dir = 'runs/detect/exp'
    video_output = os.path.join(output_dir, os.path.basename(video_path))
    crops_dir = os.path.join(output_dir, 'crops', 'cell phone')
    if os.path.exists(output_dir):
        shutil.rmtree(output_dir)  # Clear previous runs
    print(f'Running detect.py on {video_path}')
    command = ['python', 'yolov5/detect.py', '--weights', 'yolov5/yolov5s.pt', '--source', video_path, '--classes', '67', '--conf', '0.25', '--save-txt', '--save-conf', '--project', 'runs/detect', '--exist-ok', '--save-crop']
    result = subprocess.run(command, capture_output=True, text=True)
    print('detect.py has finished running')
    print(result.stdout)
    print(result.stderr)

    # Check if cell phones were detected
    cell_phones_detected = os.path.exists(crops_dir) and len(os.listdir(crops_dir)) > 0

    return cell_phones_detected, video_output

def process_video_pytorchvideo(video_path):
    # Load the pre-trained SlowFast model
    model = slowfast_r50(pretrained=True)
    model = model.eval()

    # Load the video using PyTorchVideo's EncodedVideo class
    video = EncodedVideo.from_path(video_path)

    # Select the time range to sample from the video
    start_time = 0
    end_time = min(video.duration, 15)  # Use the shorter of 15 seconds or video duration

    # Sample frames uniformly
    video_data = video.get_clip(start_sec=start_time, end_sec=end_time)
    frames = video_data["video"]

    # Define the transforms to apply to each frame
    transform = transforms.Compose([
        UniformTemporalSubsample(32),
        ShortSideScale(256),
        transforms.CenterCrop(224),
        transforms.Lambda(lambda x: x / 255.0),
    ])

    # Apply the initial transforms
    frames = transform(frames)

    # Normalize the RGB channels
    normalize = transforms.Normalize(mean=[0.45, 0.45, 0.45], std=[0.225, 0.225, 0.225])
    frames = frames.permute(1, 0, 2, 3)
    frames = normalize(frames)
    frames = frames.permute(1, 0, 2, 3)

    # Prepare the input format for SlowFast
    fast_pathway = frames
    slow_pathway = torch.index_select(
        frames,
        1,
        torch.linspace(0, frames.shape[1] - 1, frames.shape[1] // 4).long(),
    )

    # Combine pathways into a list and add a batch dimension
    inputs = [slow_pathway.unsqueeze(0), fast_pathway.unsqueeze(0)]

    # Run inference
    with torch.no_grad():
        preds = model(inputs)

    # Load Kinetics-400 class labels
    kinetics_labels = []
    with open('kinetics_400_labels.txt', 'r') as f:
        for line in f:
            kinetics_labels.append(line.strip())

    # Get top 5 predictions
    top5_pred = torch.topk(preds, k=5)
    top5_actions = []
    for i in range(5):
        action = kinetics_labels[top5_pred.indices[0][i]]
        confidence = top5_pred.values[0][i].item()
        top5_actions.append((action, confidence))

    return top5_actions

def get_video_html(video_path):
    video_file = open(video_path, 'rb')
    video_bytes = video_file.read()
    video_b64 = base64.b64encode(video_bytes).decode()
    video_html = f'<video width="320" height="240" controls><source src="data:video/mp4;base64,{video_b64}" type="video/mp4"></video>'
    return video_html

# Streamlit app
st.title('YOLOv5 Cell Phone Detection and PyTorchVideo Action Recognition')

uploaded_file = st.file_uploader('Upload a video file', type=['mp4', 'avi', 'mov'])

if uploaded_file is not None:
    video_path = f'uploaded_video.{uploaded_file.type.split("/")[1]}'
    with open(video_path, "wb") as f:
        f.write(uploaded_file.getbuffer())

    st.video(video_path)

    # YOLOv5 Object Detection
    st.write('Running YOLOv5 inference for cell phone detection...')
    try:
        cell_phones_detected, annotated_video_path = process_video_yolov5(video_path)
        st.write('YOLOv5 inference completed.')

        if cell_phones_detected:
            st.write('Cell phones detected in the video.')
            video_html = get_video_html(annotated_video_path)
            st.markdown(video_html, unsafe_allow_html=True)
        else:
            st.write('No cell phones detected in the video.')

    except Exception as e:
        st.write(f'Error during YOLOv5 inference: {e}')

    # PyTorchVideo Action Recognition
    st.write('Running PyTorchVideo inference for action recognition...')
    try:
        top5_actions = process_video_pytorchvideo(video_path)
        st.write('PyTorchVideo inference completed.')

        st.write('Top 5 detected actions:')
        for action, confidence in top5_actions:
            st.write(f"{action}: {confidence:.3f}")

    except Exception as e:
        st.write(f'Error during PyTorchVideo inference: {e}')
"""

with open("app.py", "w") as file:
    file.write(app_code)
print("app.py file has been written.")

# Start the Streamlit app in the background
import subprocess
streamlit_process = subprocess.Popen(['streamlit', 'run', 'app.py'])

# Start LocalTunnel
import time
import requests

time.sleep(5)
localtunnel_process = subprocess.Popen(['lt', '--port', '8501'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

# Capture the LocalTunnel URL
public_url = None
for line in iter(localtunnel_process.stdout.readline, b''):
    decoded_line = line.decode().strip()
    print(decoded_line)
    if 'your url is:' in decoded_line:
        public_url = decoded_line.split(' ')[-1]
        break

if public_url:
    print(f'Streamlit app running at: {public_url}')
    headers = {
        'User-Agent': 'Mozilla/5.0',
        'bypass-tunnel-reminder': '1'
    }
    response = requests.get(public_url, headers=headers)
    if response.status_code == 200:
        print(f'Access the Streamlit app at: {public_url}')
    else:
        print('Failed to bypass the password screen.')
else:
    print('Failed to create LocalTunnel.')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.7/132.7 kB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 90.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 97.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.5/33.5 MB 57.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.1 MB/s eta 0:00:00
  Created wheel for pytorchvideo: filename=pytorchvideo-0.1.5-py3-none-any.whl size=188686 sha256=6b5901a62c2d0c10046f373e1321a9362754c1614b003f49f7abfa792b8de03d
  

In [ ]:
!wget -q -O - https://loca.lt/mytunnelpassword

34.16.187.42

In [ ]:
# Install required packages
!pip install streamlit opencv-python-headless pyngrok pytorchvideo torchvision iopath torch git+https://github.com/facebookresearch/fvcore.git git+https://github.com/facebookresearch/fairscale.git
!npm install -g localtunnel

# Clone repositories
!git clone https://github.com/ultralytics/yolov5
!git clone https://github.com/facebookresearch/slowfast
!git clone https://github.com/facebookresearch/pytorchvideo

# Set up YOLOv5
%cd yolov5
!pip install -qr requirements.txt comet_ml
%cd ..

# Set up SlowFast
%cd slowfast
!python setup.py build develop
%cd ..

# Set up PyTorchVideo
%cd pytorchvideo
!pip install -e .
%cd ..

# Add SlowFast to PYTHONPATH
import os
os.environ['PYTHONPATH'] += ":/content/slowfast"

# Download Kinetics-400 labels
!wget -O /content/label_map.txt https://raw.githubusercontent.com/google-deepmind/kinetics-i3d/master/data/label_map.txt

# Create app.py
app_code = """
import streamlit as st
import os
import shutil
import subprocess
import torch
from pytorchvideo.models.hub import slowfast_r50
import torchvision.transforms as transforms
from pytorchvideo.transforms import ShortSideScale, UniformTemporalSubsample
from pytorchvideo.data.encoded_video import EncodedVideo
import cv2
import base64

# Ensure detect.py is present
assert os.path.exists('yolov5/detect.py'), 'detect.py not found. Ensure it is in the yolov5 directory.'

def process_video_yolov5(video_path):
    output_dir = 'runs/detect/exp'
    video_output = os.path.join(output_dir, os.path.basename(video_path))
    if os.path.exists(output_dir):
        shutil.rmtree(output_dir)  # Clear previous runs
    print(f'Running detect.py on {video_path}')
    command = ['python', 'yolov5/detect.py', '--weights', 'yolov5/yolov5s.pt', '--source', video_path, '--classes', '67', '--conf', '0.25', '--save-txt', '--save-conf', '--project', 'runs/detect', '--exist-ok']
    result = subprocess.run(command, capture_output=True, text=True)
    print('detect.py has finished running')
    print(result.stdout)
    print(result.stderr)
    return video_output

def process_video_pytorchvideo(video_path):
    # Load the pre-trained SlowFast model
    model = slowfast_r50(pretrained=True)
    model = model.eval()

    # Load the video using PyTorchVideo's EncodedVideo class
    video = EncodedVideo.from_path(video_path)

    # Select the time range to sample from the video
    start_time = 0
    end_time = min(video.duration, 15)  # Use the shorter of 15 seconds or video duration

    # Sample frames uniformly
    video_data = video.get_clip(start_sec=start_time, end_sec=end_time)
    frames = video_data["video"]

    # Define the transforms to apply to each frame
    transform = transforms.Compose([
        UniformTemporalSubsample(32),
        ShortSideScale(256),
        transforms.CenterCrop(224),
        transforms.Lambda(lambda x: x / 255.0),
    ])

    # Apply the initial transforms
    frames = transform(frames)

    # Normalize the RGB channels
    normalize = transforms.Normalize(mean=[0.45, 0.45, 0.45], std=[0.225, 0.225, 0.225])
    frames = frames.permute(1, 0, 2, 3)
    frames = normalize(frames)
    frames = frames.permute(1, 0, 2, 3)

    # Prepare the input format for SlowFast
    fast_pathway = frames
    slow_pathway = torch.index_select(
        frames,
        1,
        torch.linspace(0, frames.shape[1] - 1, frames.shape[1] // 4).long(),
    )

    # Combine pathways into a list and add a batch dimension
    inputs = [slow_pathway.unsqueeze(0), fast_pathway.unsqueeze(0)]

    # Run inference
    with torch.no_grad():
        preds = model(inputs)

    # Load Kinetics-400 class labels
    kinetics_labels = []
    with open('/content/label_map.txt', 'r') as f:
        kinetics_labels = [line.strip() for line in f if line.strip()]

    # Get top 5 predictions
    top5_pred = torch.topk(preds, k=5)
    top5_actions = []
    for i in range(5):
        if i < len(kinetics_labels):
            action = kinetics_labels[top5_pred.indices[0][i]]
            confidence = top5_pred.values[0][i].item()
            top5_actions.append((action, confidence))
        else:
            break

    return top5_actions

def get_video_html(path):
    video_file = open(path, 'rb')
    video_bytes = video_file.read()
    video_b64 = base64.b64encode(video_bytes).decode()
    return f'<video width="320" height="240" controls><source src="data:video/mp4;base64,{video_b64}" type="video/mp4"></video>'

# Streamlit app
st.title('YOLOv5 Cell Phone Detection and PyTorchVideo Action Recognition')

uploaded_file = st.file_uploader('Upload a video file', type=['mp4', 'avi', 'mov'])

if uploaded_file is not None:
    video_path = f'uploaded_video.{uploaded_file.type.split("/")[1]}'
    with open(video_path, "wb") as f:
        f.write(uploaded_file.getbuffer())

    st.video(video_path)

    # YOLOv5 Object Detection
    st.subheader('YOLOv5 Cell Phone Detection')
    st.write('Running YOLOv5 inference...')
    try:
        annotated_video_path = process_video_yolov5(video_path)
        st.write('YOLOv5 inference completed.')

        if os.path.exists(annotated_video_path):
            st.write('Cell phones detected in the video.')
            st.markdown(get_video_html(annotated_video_path), unsafe_allow_html=True)
        else:
            st.write('No cell phones detected in the video.')
    except Exception as e:
        st.write(f'Error during YOLOv5 inference: {e}')

    # PyTorchVideo Action Recognition
    st.subheader('PyTorchVideo Action Recognition')
    st.write('Running PyTorchVideo inference...')
    try:
        top5_actions = process_video_pytorchvideo(video_path)
        st.write('PyTorchVideo inference completed.')

        st.write('Top 5 predicted actions:')
        for i, (action, confidence) in enumerate(top5_actions, 1):
            st.write(f"{i}. {action}: {confidence:.3f}")
    except Exception as e:
        st.write(f'Error during PyTorchVideo inference: {e}')
"""

with open("app.py", "w") as file:
    file.write(app_code)
print("app.py file has been written.")

# Start the Streamlit app in the background
import subprocess
streamlit_process = subprocess.Popen(['streamlit', 'run', 'app.py'])

# Start LocalTunnel
import time
import requests

time.sleep(5)
localtunnel_process = subprocess.Popen(['lt', '--port', '8501'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

# Capture the LocalTunnel URL
public_url = None
for line in iter(localtunnel_process.stdout.readline, b''):
    decoded_line = line.decode().strip()
    print(decoded_line)
    if 'your url is:' in decoded_line:
        public_url = decoded_line.split(' ')[-1]
        break

if public_url:
    print(f'Streamlit app running at: {public_url}')
    headers = {
        'User-Agent': 'Mozilla/5.0',
        'bypass-tunnel-reminder': '1'
    }
    response = requests.get(public_url, headers=headers)
    if response.status_code == 200:
        print(f'Access the Streamlit app at: {public_url}')
    else:
        print('Failed to bypass the password screen.')
else:
    print('Failed to create LocalTunnel.')

  Cloning https://github.com/facebookresearch/fvcore.git to /tmp/pip-req-build-qcc2kb0j
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/fvcore.git /tmp/pip-req-build-qcc2kb0j
  Resolved https://github.com/facebookresearch/fvcore.git to commit f3b07ea37daff5b774702708134f7b702e2dc83e
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/facebookresearch/fairscale.git to /tmp/pip-req-build-quew4qx7
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/fairscale.git /tmp/pip-req-build-quew4qx7
  Resolved https://github.com/facebookresearch/fairscale.git to commit 5f484b3545f27eddb19d970fbe1d361b9c5f2b07
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.7/132.7 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!wget -q -O - https://loca.lt/mytunnelpassword

104.196.163.167

In [ ]:
# Install required packages
!pip install streamlit opencv-python-headless pyngrok pytorchvideo torchvision iopath torch git+https://github.com/facebookresearch/fvcore.git git+https://github.com/facebookresearch/fairscale.git
!npm install -g localtunnel

# Clone repositories
!git clone https://github.com/ultralytics/yolov5
!git clone https://github.com/facebookresearch/slowfast
!git clone https://github.com/facebookresearch/pytorchvideo

# Set up YOLOv5
%cd yolov5
!pip install -qr requirements.txt comet_ml
%cd ..

# Set up SlowFast
%cd slowfast
!python setup.py build develop
%cd ..

# Set up PyTorchVideo
%cd pytorchvideo
!pip install -e .
%cd ..

# Add SlowFast to PYTHONPATH
import os
os.environ['PYTHONPATH'] += ":/content/slowfast"

# Download Kinetics-400 labels
!wget -O /content/label_map.txt https://raw.githubusercontent.com/google-deepmind/kinetics-i3d/master/data/label_map.txt

# Create app.py
app_code = """
import streamlit as st
import os
import shutil
import subprocess
import torch
from pytorchvideo.models.hub import slowfast_r50
import torchvision.transforms as transforms
from pytorchvideo.transforms import ShortSideScale, UniformTemporalSubsample
from pytorchvideo.data.encoded_video import EncodedVideo
import cv2
import time

# Ensure detect.py is present
assert os.path.exists('yolov5/detect.py'), 'detect.py not found. Ensure it is in the yolov5 directory.'

def process_video_yolov5(video_path):
    output_dir = 'runs/detect/exp'
    video_output = os.path.join(output_dir, os.path.basename(video_path))
    if os.path.exists(output_dir):
        shutil.rmtree(output_dir)  # Clear previous runs
    print(f'Running detect.py on {video_path}')
    command = ['python', 'yolov5/detect.py', '--weights', 'yolov5/yolov5s.pt', '--source', video_path, '--classes', '67', '--conf', '0.25', '--save-txt', '--save-conf', '--project', 'runs/detect', '--exist-ok']
    result = subprocess.run(command, capture_output=True, text=True)
    print('detect.py has finished running')
    print(result.stdout)
    print(result.stderr)

    # Add a delay to ensure the file is fully written
    time.sleep(5)

    return video_output

def process_video_pytorchvideo(video_path):
    # Load the pre-trained SlowFast model
    model = slowfast_r50(pretrained=True)
    model = model.eval()

    # Load the video using PyTorchVideo's EncodedVideo class
    video = EncodedVideo.from_path(video_path)

    # Select the time range to sample from the video
    start_time = 0
    end_time = min(video.duration, 15)  # Use the shorter of 15 seconds or video duration

    # Sample frames uniformly
    video_data = video.get_clip(start_sec=start_time, end_sec=end_time)
    frames = video_data["video"]

    # Define the transforms to apply to each frame
    transform = transforms.Compose([
        UniformTemporalSubsample(32),
        ShortSideScale(256),
        transforms.CenterCrop(224),
        transforms.Lambda(lambda x: x / 255.0),
    ])

    # Apply the initial transforms
    frames = transform(frames)

    # Normalize the RGB channels
    normalize = transforms.Normalize(mean=[0.45, 0.45, 0.45], std=[0.225, 0.225, 0.225])
    frames = frames.permute(1, 0, 2, 3)
    frames = normalize(frames)
    frames = frames.permute(1, 0, 2, 3)

    # Prepare the input format for SlowFast
    fast_pathway = frames
    slow_pathway = torch.index_select(
        frames,
        1,
        torch.linspace(0, frames.shape[1] - 1, frames.shape[1] // 4).long(),
    )

    # Combine pathways into a list and add a batch dimension
    inputs = [slow_pathway.unsqueeze(0), fast_pathway.unsqueeze(0)]

    # Run inference
    with torch.no_grad():
        preds = model(inputs)

    # Load Kinetics-400 class labels
    kinetics_labels = []
    with open('/content/label_map.txt', 'r') as f:
        kinetics_labels = [line.strip() for line in f if line.strip()]

    # Get top 5 predictions
    top5_pred = torch.topk(preds, k=5)
    top5_actions = []
    for i in range(5):
        if i < len(kinetics_labels):
            action = kinetics_labels[top5_pred.indices[0][i]]
            confidence = top5_pred.values[0][i].item()
            top5_actions.append((action, confidence))
        else:
            break

    return top5_actions

# Streamlit app
st.title('YOLOv5 Cell Phone Detection and PyTorchVideo Action Recognition')

uploaded_file = st.file_uploader('Upload a video file', type=['mp4', 'avi', 'mov'])

if uploaded_file is not None:
    video_path = f'uploaded_video.{uploaded_file.type.split("/")[1]}'
    with open(video_path, "wb") as f:
        f.write(uploaded_file.getbuffer())

    st.video(video_path)

    # YOLOv5 Object Detection
    st.subheader('YOLOv5 Cell Phone Detection')
    st.write('Running YOLOv5 inference...')
    try:
        annotated_video_path = process_video_yolov5(video_path)
        st.write('YOLOv5 inference completed.')

        if os.path.exists(annotated_video_path):
            st.write('Cell phones detected in the video.')
            st.write(f'Annotated video path: {annotated_video_path}')
            try:
                st.video(annotated_video_path)
            except Exception as e:
                st.error(f"Error displaying video: {e}")
                st.write("Attempting to display video using file_uploader...")
                with open(annotated_video_path, "rb") as file:
                    st.download_button(
                        label="Download processed video",
                        data=file,
                        file_name="processed_video.mp4",
                        mime="video/mp4"
                    )
        else:
            st.write('No cell phones detected in the video or annotated video not found.')
            st.write(f'Expected video path: {annotated_video_path}')
    except Exception as e:
        st.error(f'Error during YOLOv5 inference: {e}')

    # PyTorchVideo Action Recognition
    st.subheader('PyTorchVideo Action Recognition')
    st.write('Running PyTorchVideo inference...')
    try:
        top5_actions = process_video_pytorchvideo(video_path)
        st.write('PyTorchVideo inference completed.')

        st.write('Top 5 predicted actions:')
        for i, (action, confidence) in enumerate(top5_actions, 1):
            st.write(f"{i}. {action}: {confidence:.3f}")
    except Exception as e:
        st.error(f'Error during PyTorchVideo inference: {e}')
"""

with open("app.py", "w") as file:
    file.write(app_code)
print("app.py file has been written.")

# Start the Streamlit app in the background
import subprocess
streamlit_process = subprocess.Popen(['streamlit', 'run', 'app.py'])

# Start LocalTunnel
import time
import requests

time.sleep(5)
localtunnel_process = subprocess.Popen(['lt', '--port', '8501'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

# Capture the LocalTunnel URL
public_url = None
for line in iter(localtunnel_process.stdout.readline, b''):
    decoded_line = line.decode().strip()
    print(decoded_line)
    if 'your url is:' in decoded_line:
        public_url = decoded_line.split(' ')[-1]
        break

if public_url:
    print(f'Streamlit app running at: {public_url}')
    headers = {
        'User-Agent': 'Mozilla/5.0',
        'bypass-tunnel-reminder': '1'
    }
    response = requests.get(public_url, headers=headers)
    if response.status_code == 200:
        print(f'Access the Streamlit app at: {public_url}')
    else:
        print('Failed to bypass the password screen.')
else:
    print('Failed to create LocalTunnel.')

  Cloning https://github.com/facebookresearch/fvcore.git to /tmp/pip-req-build-jm2ewx8e
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/fvcore.git /tmp/pip-req-build-jm2ewx8e
  Resolved https://github.com/facebookresearch/fvcore.git to commit f3b07ea37daff5b774702708134f7b702e2dc83e
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/facebookresearch/fairscale.git to /tmp/pip-req-build-hbvd_juq
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/fairscale.git /tmp/pip-req-build-hbvd_juq
  Resolved https://github.com/facebookresearch/fairscale.git to commit 5f484b3545f27eddb19d970fbe1d361b9c5f2b07
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.7/132.7 kB 9.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!wget -q -O - https://loca.lt/mytunnelpassword

35.240.200.50

In [ ]:
# Install required packages
!pip install streamlit opencv-python-headless pyngrok pytorchvideo torchvision iopath torch git+https://github.com/facebookresearch/fvcore.git git+https://github.com/facebookresearch/fairscale.git
!npm install -g localtunnel

# Clone repositories
!git clone https://github.com/ultralytics/yolov5
!git clone https://github.com/facebookresearch/slowfast
!git clone https://github.com/facebookresearch/pytorchvideo

# Set up YOLOv5
%cd yolov5
!pip install -qr requirements.txt comet_ml
%cd ..

# Set up SlowFast
%cd slowfast
!python setup.py build develop
%cd ..

# Set up PyTorchVideo
%cd pytorchvideo
!pip install -e .
%cd ..

# Add SlowFast to PYTHONPATH
import os
os.environ['PYTHONPATH'] += ":/content/slowfast"

# Download Kinetics-400 labels
!wget -O /content/label_map.txt https://raw.githubusercontent.com/google-deepmind/kinetics-i3d/master/data/label_map.txt

# Create app.py
app_code = """
import streamlit as st
import os
import shutil
import subprocess
import torch
from pytorchvideo.models.hub import slowfast_r50
import torchvision.transforms as transforms
from pytorchvideo.transforms import ShortSideScale, UniformTemporalSubsample
from pytorchvideo.data.encoded_video import EncodedVideo
import cv2
import time

# Ensure detect.py is present
assert os.path.exists('/content/yolov5/detect.py'), 'detect.py not found. Ensure it is in the yolov5 directory.'

def process_video_yolov5(video_path):
    output_dir = '/content/runs/detect/exp'
    video_output = os.path.join(output_dir, os.path.basename(video_path))
    if os.path.exists(output_dir):
        shutil.rmtree(output_dir)  # Clear previous runs
    print(f'Running detect.py on {video_path}')
    command = ['python', '/content/yolov5/detect.py', '--weights', '/content/yolov5/yolov5s.pt', '--source', video_path, '--classes', '67', '--conf', '0.25', '--save-txt', '--save-conf', '--project', '/content/runs/detect', '--exist-ok']
    result = subprocess.run(command, capture_output=True, text=True)
    print('detect.py has finished running')
    print(result.stdout)
    print(result.stderr)

    # Add a delay to ensure the file is fully written
    time.sleep(5)

    return video_output

def process_video_pytorchvideo(video_path):
    # Load the pre-trained SlowFast model
    model = slowfast_r50(pretrained=True)
    model = model.eval()

    # Load the video using PyTorchVideo's EncodedVideo class
    video = EncodedVideo.from_path(video_path)

    # Select the time range to sample from the video
    start_time = 0
    end_time = min(video.duration, 15)  # Use the shorter of 15 seconds or video duration

    # Sample frames uniformly
    video_data = video.get_clip(start_sec=start_time, end_sec=end_time)
    frames = video_data["video"]

    # Define the transforms to apply to each frame
    transform = transforms.Compose([
        UniformTemporalSubsample(32),
        ShortSideScale(256),
        transforms.CenterCrop(224),
        transforms.Lambda(lambda x: x / 255.0),
    ])

    # Apply the initial transforms
    frames = transform(frames)

    # Normalize the RGB channels
    normalize = transforms.Normalize(mean=[0.45, 0.45, 0.45], std=[0.225, 0.225, 0.225])
    frames = frames.permute(1, 0, 2, 3)
    frames = normalize(frames)
    frames = frames.permute(1, 0, 2, 3)

    # Prepare the input format for SlowFast
    fast_pathway = frames
    slow_pathway = torch.index_select(
        frames,
        1,
        torch.linspace(0, frames.shape[1] - 1, frames.shape[1] // 4).long(),
    )

    # Combine pathways into a list and add a batch dimension
    inputs = [slow_pathway.unsqueeze(0), fast_pathway.unsqueeze(0)]

    # Run inference
    with torch.no_grad():
        preds = model(inputs)

    # Load Kinetics-400 class labels
    kinetics_labels = []
    with open('/content/label_map.txt', 'r') as f:
        kinetics_labels = [line.strip() for line in f if line.strip()]

    # Get top 5 predictions
    top5_pred = torch.topk(preds, k=5)
    top5_actions = []
    for i in range(5):
        if i < len(kinetics_labels):
            action = kinetics_labels[top5_pred.indices[0][i]]
            confidence = top5_pred.values[0][i].item()
            top5_actions.append((action, confidence))
        else:
            break

    return top5_actions

# Streamlit app
st.title('YOLOv5 Cell Phone Detection and PyTorchVideo Action Recognition')

uploaded_file = st.file_uploader('Upload a video file', type=['mp4', 'avi', 'mov'])

if uploaded_file is not None:
    video_path = f'/content/uploaded_video.{uploaded_file.type.split("/")[1]}'
    with open(video_path, "wb") as f:
        f.write(uploaded_file.getbuffer())

    st.video(video_path)

    # YOLOv5 Object Detection
    st.subheader('YOLOv5 Cell Phone Detection')
    st.write('Running YOLOv5 inference...')
    try:
        annotated_video_path = process_video_yolov5(video_path)
        st.write('YOLOv5 inference completed.')

        if os.path.exists(annotated_video_path):
            st.write('Cell phones detected in the video.')
            st.write(f'Annotated video path: {annotated_video_path}')

            # Add another 5-second delay before attempting to display the video
            time.sleep(5)

            try:
                st.video(annotated_video_path)
            except Exception as e:
                st.error(f"Error displaying video: {e}")

            # Always show the download button
            st.write("You can also download the processed video:")
            with open(annotated_video_path, "rb") as file:
                st.download_button(
                    label="Download processed video",
                    data=file,
                    file_name="processed_video.mp4",
                    mime="video/mp4"
                )
        else:
            st.write('No cell phones detected in the video or annotated video not found.')
            st.write(f'Expected video path: {annotated_video_path}')
    except Exception as e:
        st.error(f'Error during YOLOv5 inference: {e}')

    # PyTorchVideo Action Recognition
    st.subheader('PyTorchVideo Action Recognition')
    st.write('Running PyTorchVideo inference...')
    try:
        top5_actions = process_video_pytorchvideo(video_path)
        st.write('PyTorchVideo inference completed.')

        st.write('Top 5 predicted actions:')
        for i, (action, confidence) in enumerate(top5_actions, 1):
            st.write(f"{i}. {action}: {confidence:.3f}")
    except Exception as e:
        st.error(f'Error during PyTorchVideo inference: {e}')
"""

with open("app.py", "w") as file:
    file.write(app_code)
print("app.py file has been written.")

# Start the Streamlit app in the background
import subprocess
streamlit_process = subprocess.Popen(['streamlit', 'run', 'app.py'])

# Start LocalTunnel
import time
import requests

time.sleep(5)
localtunnel_process = subprocess.Popen(['lt', '--port', '8501'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

# Capture the LocalTunnel URL
public_url = None
for line in iter(localtunnel_process.stdout.readline, b''):
    decoded_line = line.decode().strip()
    print(decoded_line)
    if 'your url is:' in decoded_line:
        public_url = decoded_line.split(' ')[-1]
        break

if public_url:
    print(f'Streamlit app running at: {public_url}')
    headers = {
        'User-Agent': 'Mozilla/5.0',
        'bypass-tunnel-reminder': '1'
    }
    response = requests.get(public_url, headers=headers)
    if response.status_code == 200:
        print(f'Access the Streamlit app at: {public_url}')
    else:
        print('Failed to bypass the password screen.')
else:
    print('Failed to create LocalTunnel.')

  Cloning https://github.com/facebookresearch/fvcore.git to /tmp/pip-req-build-xfbhxkzh
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/fvcore.git /tmp/pip-req-build-xfbhxkzh
  Resolved https://github.com/facebookresearch/fvcore.git to commit f3b07ea37daff5b774702708134f7b702e2dc83e
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/facebookresearch/fairscale.git to /tmp/pip-req-build-ab0lloz8
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/fairscale.git /tmp/pip-req-build-ab0lloz8
  Resolved https://github.com/facebookresearch/fairscale.git to commit 5f484b3545f27eddb19d970fbe1d361b9c5f2b07
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.7/132.7 kB 5.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!wget -q -O - https://loca.lt/mytunnelpassword

34.16.181.192

In [ ]:
# Install required packages
!pip install streamlit opencv-python-headless pyngrok pytorchvideo torchvision iopath torch git+https://github.com/facebookresearch/fvcore.git git+https://github.com/facebookresearch/fairscale.git
!npm install -g localtunnel

# Clone repositories
!git clone https://github.com/ultralytics/yolov5
!git clone https://github.com/facebookresearch/slowfast
!git clone https://github.com/facebookresearch/pytorchvideo

# Set up YOLOv5
%cd yolov5
!pip install -qr requirements.txt comet_ml
%cd ..

# Set up SlowFast
%cd slowfast
!python setup.py build develop
%cd ..

# Set up PyTorchVideo
%cd pytorchvideo
!pip install -e .
%cd ..

# Add SlowFast to PYTHONPATH
import os
os.environ['PYTHONPATH'] += ":/content/slowfast"

# Download Kinetics-400 labels
!wget -O /content/label_map.txt https://raw.githubusercontent.com/google-deepmind/kinetics-i3d/master/data/label_map.txt

# Create app.py
app_code = """
import streamlit as st
import os
import shutil
import subprocess
import torch
from pytorchvideo.models.hub import slowfast_r50
import torchvision.transforms as transforms
from pytorchvideo.transforms import ShortSideScale, UniformTemporalSubsample
from pytorchvideo.data.encoded_video import EncodedVideo
import cv2
import time
import io

# Ensure detect.py is present
assert os.path.exists('/content/yolov5/detect.py'), 'detect.py not found. Ensure it is in the yolov5 directory.'

def process_video_yolov5(video_path):
    output_dir = '/content/runs/detect/exp'
    video_output = os.path.join(output_dir, os.path.basename(video_path))
    if os.path.exists(output_dir):
        shutil.rmtree(output_dir)  # Clear previous runs
    print(f'Running detect.py on {video_path}')
    command = ['python', '/content/yolov5/detect.py', '--weights', '/content/yolov5/yolov5s.pt', '--source', video_path, '--classes', '67', '--conf', '0.25', '--save-txt', '--save-conf', '--project', '/content/runs/detect', '--exist-ok']
    result = subprocess.run(command, capture_output=True, text=True)
    print('detect.py has finished running')
    print(result.stdout)
    print(result.stderr)

    # Add a delay to ensure the file is fully written
    time.sleep(5)

    return video_output

def process_video_pytorchvideo(video_path):
    # Load the pre-trained SlowFast model
    model = slowfast_r50(pretrained=True)
    model = model.eval()

    # Load the video using PyTorchVideo's EncodedVideo class
    video = EncodedVideo.from_path(video_path)

    # Select the time range to sample from the video
    start_time = 0
    end_time = min(video.duration, 15)  # Use the shorter of 15 seconds or video duration

    # Sample frames uniformly
    video_data = video.get_clip(start_sec=start_time, end_sec=end_time)
    frames = video_data["video"]

    # Define the transforms to apply to each frame
    transform = transforms.Compose([
        UniformTemporalSubsample(32),
        ShortSideScale(256),
        transforms.CenterCrop(224),
        transforms.Lambda(lambda x: x / 255.0),
    ])

    # Apply the initial transforms
    frames = transform(frames)

    # Normalize the RGB channels
    normalize = transforms.Normalize(mean=[0.45, 0.45, 0.45], std=[0.225, 0.225, 0.225])
    frames = frames.permute(1, 0, 2, 3)
    frames = normalize(frames)
    frames = frames.permute(1, 0, 2, 3)

    # Prepare the input format for SlowFast
    fast_pathway = frames
    slow_pathway = torch.index_select(
        frames,
        1,
        torch.linspace(0, frames.shape[1] - 1, frames.shape[1] // 4).long(),
    )

    # Combine pathways into a list and add a batch dimension
    inputs = [slow_pathway.unsqueeze(0), fast_pathway.unsqueeze(0)]

    # Run inference
    with torch.no_grad():
        preds = model(inputs)

    # Load Kinetics-400 class labels
    kinetics_labels = []
    with open('/content/label_map.txt', 'r') as f:
        kinetics_labels = [line.strip() for line in f if line.strip()]

    # Get top 5 predictions
    top5_pred = torch.topk(preds, k=5)
    top5_actions = []
    for i in range(5):
        if i < len(kinetics_labels):
            action = kinetics_labels[top5_pred.indices[0][i]]
            confidence = top5_pred.values[0][i].item()
            top5_actions.append((action, confidence))
        else:
            break

    return top5_actions

# Streamlit app
st.title('YOLOv5 Cell Phone Detection and PyTorchVideo Action Recognition')

uploaded_file = st.file_uploader('Upload a video file', type=['mp4', 'avi', 'mov'])

if uploaded_file is not None:
    video_path = f'/content/uploaded_video.{uploaded_file.type.split("/")[1]}'
    with open(video_path, "wb") as f:
        f.write(uploaded_file.getbuffer())

    st.video(uploaded_file)

    # YOLOv5 Object Detection
    st.subheader('YOLOv5 Cell Phone Detection')
    st.write('Running YOLOv5 inference...')
    try:
        annotated_video_path = process_video_yolov5(video_path)
        st.write('YOLOv5 inference completed.')

        if os.path.exists(annotated_video_path):
            st.write('Cell phones detected in the video.')
            st.write(f'Annotated video path: {annotated_video_path}')

            # Add another 5-second delay before attempting to display the video
            time.sleep(5)

            # Read the video file into a BytesIO object
            with open(annotated_video_path, "rb") as file:
                video_bytes = file.read()

            # Display the video using st.video()
            st.video(video_bytes)

            # Provide a download button for the processed video
            st.write("You can also download the processed video:")
            st.download_button(
                label="Download processed video",
                data=video_bytes,
                file_name="processed_video.mp4",
                mime="video/mp4"
            )
        else:
            st.write('No cell phones detected in the video or annotated video not found.')
            st.write(f'Expected video path: {annotated_video_path}')
    except Exception as e:
        st.error(f'Error during YOLOv5 inference: {e}')

    # PyTorchVideo Action Recognition
    st.subheader('PyTorchVideo Action Recognition')
    st.write('Running PyTorchVideo inference...')
    try:
        top5_actions = process_video_pytorchvideo(video_path)
        st.write('PyTorchVideo inference completed.')

        st.write('Top 5 predicted actions:')
        for i, (action, confidence) in enumerate(top5_actions, 1):
            st.write(f"{i}. {action}: {confidence:.3f}")
    except Exception as e:
        st.error(f'Error during PyTorchVideo inference: {e}')
"""

with open("app.py", "w") as file:
    file.write(app_code)
print("app.py file has been written.")

# Start the Streamlit app in the background
import subprocess
streamlit_process = subprocess.Popen(['streamlit', 'run', 'app.py'])

# Start LocalTunnel
import time
import requests

time.sleep(5)
localtunnel_process = subprocess.Popen(['lt', '--port', '8501'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

# Capture the LocalTunnel URL
public_url = None
for line in iter(localtunnel_process.stdout.readline, b''):
    decoded_line = line.decode().strip()
    print(decoded_line)
    if 'your url is:' in decoded_line:
        public_url = decoded_line.split(' ')[-1]
        break

if public_url:
    print(f'Streamlit app running at: {public_url}')
    headers = {
        'User-Agent': 'Mozilla/5.0',
        'bypass-tunnel-reminder': '1'
    }
    response = requests.get(public_url, headers=headers)
    if response.status_code == 200:
        print(f'Access the Streamlit app at: {public_url}')
    else:
        print('Failed to bypass the password screen.')
else:
    print('Failed to create LocalTunnel.')

  Cloning https://github.com/facebookresearch/fvcore.git to /tmp/pip-req-build-kcr74mey
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/fvcore.git /tmp/pip-req-build-kcr74mey
  Resolved https://github.com/facebookresearch/fvcore.git to commit f3b07ea37daff5b774702708134f7b702e2dc83e
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/facebookresearch/fairscale.git to /tmp/pip-req-build-2_iuo5v4
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/fairscale.git /tmp/pip-req-build-2_iuo5v4
  Resolved https://github.com/facebookresearch/fairscale.git to commit 5f484b3545f27eddb19d970fbe1d361b9c5f2b07
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.7/132.7 kB 7.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!wget -q -O - https://loca.lt/mytunnelpassword

35.204.244.179

In [ ]:
# Install required packages
!pip install streamlit opencv-python-headless pyngrok pytorchvideo torchvision iopath torch git+https://github.com/facebookresearch/fvcore.git git+https://github.com/facebookresearch/fairscale.git
!npm install -g localtunnel

# Clone repositories
!git clone https://github.com/ultralytics/yolov5
!git clone https://github.com/facebookresearch/slowfast
!git clone https://github.com/facebookresearch/pytorchvideo

# Set up YOLOv5
%cd yolov5
!pip install -qr requirements.txt comet_ml
%cd ..

# Set up SlowFast
%cd slowfast
!python setup.py build develop
%cd ..

# Set up PyTorchVideo
%cd pytorchvideo
!pip install -e .
%cd ..

# Add SlowFast to PYTHONPATH
import os
os.environ['PYTHONPATH'] += ":/content/slowfast"

# Download Kinetics-400 labels
!wget -O /content/label_map.txt https://raw.githubusercontent.com/google-deepmind/kinetics-i3d/master/data/label_map.txt

# Create app.py
app_code = """
import streamlit as st
import os
import shutil
import subprocess
import torch
from pytorchvideo.models.hub import slowfast_r50
import torchvision.transforms as transforms
from pytorchvideo.transforms import ShortSideScale, UniformTemporalSubsample
from pytorchvideo.data.encoded_video import EncodedVideo
import cv2
import time
import io
from PIL import Image

# Ensure detect.py is present
assert os.path.exists('/content/yolov5/detect.py'), 'detect.py not found. Ensure it is in the yolov5 directory.'

def process_video_yolov5(video_path):
    output_dir = '/content/runs/detect/exp'
    video_output = os.path.join(output_dir, os.path.basename(video_path))
    crops_dir = os.path.join(output_dir, 'crops', 'cell phone')
    if os.path.exists(output_dir):
        shutil.rmtree(output_dir)  # Clear previous runs
    print(f'Running detect.py on {video_path}')
    command = ['python', '/content/yolov5/detect.py', '--weights', '/content/yolov5/yolov5s.pt', '--source', video_path, '--classes', '67', '--conf', '0.25', '--save-txt', '--save-conf', '--project', '/content/runs/detect', '--exist-ok', '--save-crop']
    result = subprocess.run(command, capture_output=True, text=True)
    print('detect.py has finished running')
    print(result.stdout)
    print(result.stderr)

    # Add a delay to ensure the file is fully written
    time.sleep(5)

    # Gather detected frames
    detected_frames = []
    if os.path.exists(crops_dir):
        for file in os.listdir(crops_dir):
            if file.endswith(('.jpg', '.jpeg', '.png')):
                img_path = os.path.join(crops_dir, file)
                img = Image.open(img_path)
                detected_frames.append(img)

    return video_output, detected_frames

def create_collage(frames, cols=5):
    if len(frames) == 0:
        return None

    # Determine the size of the collage
    frame_width, frame_height = frames[0].size
    rows = (len(frames) + cols - 1) // cols

    collage_width = cols * frame_width
    collage_height = rows * frame_height

    # Create a blank collage
    collage = Image.new('RGB', (collage_width, collage_height))

    # Fill the collage with frames
    for idx, frame in enumerate(frames):
        row = idx // cols
        col = idx % cols
        collage.paste(frame, (col * frame_width, row * frame_height))

    return collage

def process_video_pytorchvideo(video_path):
    # Load the pre-trained SlowFast model
    model = slowfast_r50(pretrained=True)
    model = model.eval()

    # Load the video using PyTorchVideo's EncodedVideo class
    video = EncodedVideo.from_path(video_path)

    # Select the time range to sample from the video
    start_time = 0
    end_time = min(video.duration, 15)  # Use the shorter of 15 seconds or video duration

    # Sample frames uniformly
    video_data = video.get_clip(start_sec=start_time, end_sec=end_time)
    frames = video_data["video"]

    # Define the transforms to apply to each frame
    transform = transforms.Compose([
        UniformTemporalSubsample(32),
        ShortSideScale(256),
        transforms.CenterCrop(224),
        transforms.Lambda(lambda x: x / 255.0),
    ])

    # Apply the initial transforms
    frames = transform(frames)

    # Normalize the RGB channels
    normalize = transforms.Normalize(mean=[0.45, 0.45, 0.45], std=[0.225, 0.225, 0.225])
    frames = frames.permute(1, 0, 2, 3)
    frames = normalize(frames)
    frames = frames.permute(1, 0, 2, 3)

    # Prepare the input format for SlowFast
    fast_pathway = frames
    slow_pathway = torch.index_select(
        frames,
        1,
        torch.linspace(0, frames.shape[1] - 1, frames.shape[1] // 4).long(),
    )

    # Combine pathways into a list and add a batch dimension
    inputs = [slow_pathway.unsqueeze(0), fast_pathway.unsqueeze(0)]

    # Run inference
    with torch.no_grad():
        preds = model(inputs)

    # Load Kinetics-400 class labels
    kinetics_labels = []
    with open('/content/label_map.txt', 'r') as f:
        kinetics_labels = [line.strip() for line in f if line.strip()]

    # Get top 5 predictions
    top5_pred = torch.topk(preds, k=5)
    top5_actions = []
    for i in range(5):
        if i < len(kinetics_labels):
            action = kinetics_labels[top5_pred.indices[0][i]]
            confidence = top5_pred.values[0][i].item()
            top5_actions.append((action, confidence))
        else:
            break

    return top5_actions

# Streamlit app
st.title('YOLOv5 Cell Phone Detection and PyTorchVideo Action Recognition')

uploaded_file = st.file_uploader('Upload a video file', type=['mp4', 'avi', 'mov'])

if uploaded_file is not None:
    video_path = f'/content/uploaded_video.{uploaded_file.type.split("/")[1]}'
    with open(video_path, "wb") as f:
        f.write(uploaded_file.getbuffer())

    st.video(uploaded_file)

    # YOLOv5 Object Detection
    st.subheader('YOLOv5 Cell Phone Detection')
    st.write('Running YOLOv5 inference...')
    try:
        annotated_video_path, detected_frames = process_video_yolov5(video_path)
        st.write('YOLOv5 inference completed.')

        if detected_frames:
            st.write(f'True Positive: {len(detected_frames)} frames detected with cell phones')
            collage_image = create_collage(detected_frames)
            if collage_image:
                st.image(collage_image, caption='Collage of detected frames')
        else:
            st.write('False Positive: No frames with cell phones detected')

        if os.path.exists(annotated_video_path):
            # Provide a download button for the processed video
            with open(annotated_video_path, "rb") as file:
                video_bytes = file.read()
            st.write("You can download the processed video:")
            st.download_button(
                label="Download processed video",
                data=video_bytes,
                file_name="processed_video.mp4",
                mime="video/mp4"
            )
        else:
            st.write('Annotated video not found.')
            st.write(f'Expected video path: {annotated_video_path}')
    except Exception as e:
        st.error(f'Error during YOLOv5 inference: {e}')

    # PyTorchVideo Action Recognition
    st.subheader('PyTorchVideo Action Recognition')
    st.write('Running PyTorchVideo inference...')
    try:
        top5_actions = process_video_pytorchvideo(video_path)
        st.write('PyTorchVideo inference completed.')

        st.write('Top 5 predicted actions:')
        for i, (action, confidence) in enumerate(top5_actions, 1):
            st.write(f"{i}. {action}: {confidence:.3f}")
    except Exception as e:
        st.error(f'Error during PyTorchVideo inference: {e}')
"""

with open("app.py", "w") as file:
    file.write(app_code)
print("app.py file has been written.")

# Start the Streamlit app in the background
import subprocess
streamlit_process = subprocess.Popen(['streamlit', 'run', 'app.py'])

# Start LocalTunnel
import time
import requests

time.sleep(5)
localtunnel_process = subprocess.Popen(['lt', '--port', '8501'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

# Capture the LocalTunnel URL
public_url = None
for line in iter(localtunnel_process.stdout.readline, b''):
    decoded_line = line.decode().strip()
    print(decoded_line)
    if 'your url is:' in decoded_line:
        public_url = decoded_line.split(' ')[-1]
        break

if public_url:
    print(f'Streamlit app running at: {public_url}')
    headers = {
        'User-Agent': 'Mozilla/5.0',
        'bypass-tunnel-reminder': '1'
    }
    response = requests.get(public_url, headers=headers)
    if response.status_code == 200:
        print(f'Access the Streamlit app at: {public_url}')
    else:
        print('Failed to bypass the password screen.')
else:
    print('Failed to create LocalTunnel.')

  Cloning https://github.com/facebookresearch/fvcore.git to /tmp/pip-req-build-udycw_t4
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/fvcore.git /tmp/pip-req-build-udycw_t4
  Resolved https://github.com/facebookresearch/fvcore.git to commit f3b07ea37daff5b774702708134f7b702e2dc83e
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/facebookresearch/fairscale.git to /tmp/pip-req-build-x0m04wfw
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/fairscale.git /tmp/pip-req-build-x0m04wfw
  Resolved https://github.com/facebookresearch/fairscale.git to commit 5f484b3545f27eddb19d970fbe1d361b9c5f2b07
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.7/132.7 kB 10.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!wget -q -O - https://loca.lt/mytunnelpassword

35.240.145.80

In [ ]:
# Install required packages
!pip install streamlit opencv-python-headless pyngrok pytorchvideo torchvision iopath torch git+https://github.com/facebookresearch/fvcore.git git+https://github.com/facebookresearch/fairscale.git
!npm install -g localtunnel

# Clone repositories
!git clone https://github.com/ultralytics/yolov5
!git clone https://github.com/facebookresearch/slowfast
!git clone https://github.com/facebookresearch/pytorchvideo

# Set up YOLOv5
%cd yolov5
!pip install -qr requirements.txt comet_ml
%cd ..

# Set up SlowFast
%cd slowfast
!python setup.py build develop
%cd ..

# Set up PyTorchVideo
%cd pytorchvideo
!pip install -e .
%cd ..

# Add SlowFast to PYTHONPATH
import os
os.environ['PYTHONPATH'] += ":/content/slowfast"

# Download Kinetics-400 labels
!wget -O /content/label_map.txt https://raw.githubusercontent.com/google-deepmind/kinetics-i3d/master/data/label_map.txt

# Create app.py
app_code = """
import streamlit as st
import os
import shutil
import subprocess
import torch
from pytorchvideo.models.hub import slowfast_r50
from pytorchvideo.models.hub import slowfast_r101
import torchvision.transforms as transforms
from pytorchvideo.transforms import ShortSideScale, UniformTemporalSubsample
from pytorchvideo.data.encoded_video import EncodedVideo
import cv2
import time
import io
from PIL import Image
# Ensure detect.py is present
assert os.path.exists('/content/yolov5/detect.py'), 'detect.py not found. Ensure it is in the yolov5 directory.'

def process_video_yolov5(video_path):
    output_dir = '/content/runs/detect/exp'
    video_output = os.path.join(output_dir, os.path.basename(video_path))
    crops_dir = os.path.join(output_dir, 'crops', 'cell phone')
    if os.path.exists(output_dir):
        shutil.rmtree(output_dir)  # Clear previous runs
    print(f'Running detect.py on {video_path}')
    command = ['python', '/content/yolov5/detect.py', '--weights', '/content/yolov5/yolov5s.pt', '--source', video_path, '--classes', '67', '--conf', '0.25', '--save-txt', '--save-conf', '--project', '/content/runs/detect', '--exist-ok', '--save-crop']
    result = subprocess.run(command, capture_output=True, text=True)
    print('detect.py has finished running')
    print(result.stdout)
    print(result.stderr)

    # Add a delay to ensure the file is fully written
    time.sleep(5)

    # Gather detected frames
    detected_frames = []
    if os.path.exists(crops_dir):
        for file in os.listdir(crops_dir):
            if file.endswith(('.jpg', '.jpeg', '.png')):
                img_path = os.path.join(crops_dir, file)
                img = Image.open(img_path)
                detected_frames.append(img)

    return video_output, detected_frames

def create_collage(frames, cols=5):
    if len(frames) == 0:
        return None

    # Determine the size of the collage
    frame_width, frame_height = frames[0].size
    rows = (len(frames) + cols - 1) // cols

    collage_width = cols * frame_width
    collage_height = rows * frame_height

    # Create a blank collage
    collage = Image.new('RGB', (collage_width, collage_height))

    # Fill the collage with frames
    for idx, frame in enumerate(frames):
        row = idx // cols
        col = idx % cols
        collage.paste(frame, (col * frame_width, row * frame_height))

    return collage

def process_video_pytorchvideo(video_path):
    # Load the pre-trained SlowFast model
    model = slowfast_r101(pretrained="kinetics_600")
    model = model.eval()

    # Load the video using PyTorchVideo's EncodedVideo class
    video = EncodedVideo.from_path(video_path)

    # Select the time range to sample from the video
    start_time = 0
    end_time = min(video.duration, 15)  # Use the shorter of 15 seconds or video duration

    # Sample frames uniformly
    video_data = video.get_clip(start_sec=start_time, end_sec=end_time)
    frames = video_data["video"]

    # Define the transforms to apply to each frame
    transform = transforms.Compose([
        UniformTemporalSubsample(32),
        ShortSideScale(256),
        transforms.CenterCrop(224),
        transforms.Lambda(lambda x: x / 255.0),
    ])

    # Apply the initial transforms
    frames = transform(frames)

    # Normalize the RGB channels
    normalize = transforms.Normalize(mean=[0.45, 0.45, 0.45], std=[0.225, 0.225, 0.225])
    frames = frames.permute(1, 0, 2, 3)
    frames = normalize(frames)
    frames = frames.permute(1, 0, 2, 3)

    # Prepare the input format for SlowFast
    fast_pathway = frames
    slow_pathway = torch.index_select(
        frames,
        1,
        torch.linspace(0, frames.shape[1] - 1, frames.shape[1] // 4).long(),
    )

    # Combine pathways into a list and add a batch dimension
    inputs = [slow_pathway.unsqueeze(0), fast_pathway.unsqueeze(0)]

    # Run inference
    with torch.no_grad():
        preds = model(inputs)

    # Load Kinetics-400 class labels
    kinetics_labels = []
    with open('/content/label_map.txt', 'r') as f:
        kinetics_labels = [line.strip() for line in f if line.strip()]

    # Get top 5 predictions
    top5_pred = torch.topk(preds, k=5)
    top5_actions = []
    for i in range(5):
        if i < len(kinetics_labels):
            action = kinetics_labels[top5_pred.indices[0][i]]
            confidence = top5_pred.values[0][i].item()
            top5_actions.append((action, confidence))
        else:
            break

    return top5_actions

# Streamlit app
st.title('YOLOv5 Cell Phone Detection and PyTorchVideo Action Recognition')

uploaded_file = st.file_uploader('Upload a video file', type=['mp4', 'avi', 'mov'])

if uploaded_file is not None:
    video_path = f'/content/uploaded_video.{uploaded_file.type.split("/")[1]}'
    with open(video_path, "wb") as f:
        f.write(uploaded_file.getbuffer())

    st.video(uploaded_file)

    # YOLOv5 Object Detection
    st.subheader('YOLOv5 Cell Phone Detection')
    st.write('Running YOLOv5 inference...')
    try:
        annotated_video_path, detected_frames = process_video_yolov5(video_path)
        st.write('YOLOv5 inference completed.')

        if detected_frames:
            st.write(f'True Positive: {len(detected_frames)} frames detected with cell phones')
            collage_image = create_collage(detected_frames)
            if collage_image:
                st.image(collage_image, caption='Collage of detected frames')
        else:
            st.write('False Positive: No frames with cell phones detected')

        if os.path.exists(annotated_video_path):
            # Provide a download button for the processed video
            with open(annotated_video_path, "rb") as file:
                video_bytes = file.read()
            st.write("You can download the processed video:")
            st.download_button(
                label="Download processed video",
                data=video_bytes,
                file_name="processed_video.mp4",
                mime="video/mp4"
            )
        else:
            st.write('Annotated video not found.')
            st.write(f'Expected video path: {annotated_video_path}')
    except Exception as e:
        st.error(f'Error during YOLOv5 inference: {e}')

    # PyTorchVideo Action Recognition
    st.subheader('PyTorchVideo Action Recognition')
    st.write('Running PyTorchVideo inference...')
    try:
        top5_actions = process_video_pytorchvideo(video_path)
        st.write('PyTorchVideo inference completed.')

        st.write('Top 5 predicted actions:')
        for i, (action, confidence) in enumerate(top5_actions, 1):
            st.write(f"{i}. {action}: {confidence:.3f}")
    except Exception as e:
        st.error(f'Error during PyTorchVideo inference: {e}')
"""

with open("app.py", "w") as file:
    file.write(app_code)
print("app.py file has been written.")

# Start the Streamlit app in the background
import subprocess
streamlit_process = subprocess.Popen(['streamlit', 'run', 'app.py'])

# Start LocalTunnel
import time
import requests

time.sleep(5)
localtunnel_process = subprocess.Popen(['lt', '--port', '8501'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

# Capture the LocalTunnel URL
public_url = None
for line in iter(localtunnel_process.stdout.readline, b''):
    decoded_line = line.decode().strip()
    print(decoded_line)
    if 'your url is:' in decoded_line:
        public_url = decoded_line.split(' ')[-1]
        break

if public_url:
    print(f'Streamlit app running at: {public_url}')
    headers = {
        'User-Agent': 'Mozilla/5.0',
        'bypass-tunnel-reminder': '1'
    }
    response = requests.get(public_url, headers=headers)
    if response.status_code == 200:
        print(f'Access the Streamlit app at: {public_url}')
    else:
        print('Failed to bypass the password screen.')
else:
    print('Failed to create LocalTunnel.')

  Cloning https://github.com/facebookresearch/fvcore.git to /tmp/pip-req-build-ec_gvcj9
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/fvcore.git /tmp/pip-req-build-ec_gvcj9
  Resolved https://github.com/facebookresearch/fvcore.git to commit f3b07ea37daff5b774702708134f7b702e2dc83e
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/facebookresearch/fairscale.git to /tmp/pip-req-build-fkytal08
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/fairscale.git /tmp/pip-req-build-fkytal08
  Resolved https://github.com/facebookresearch/fairscale.git to commit 5f484b3545f27eddb19d970fbe1d361b9c5f2b07
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.7/132.7 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!wget -q -O - https://loca.lt/mytunnelpassword

34.125.30.86

In [ ]:
# Install required packages
!pip install streamlit opencv-python-headless pyngrok pytorchvideo torchvision iopath torch git+https://github.com/facebookresearch/fvcore.git git+https://github.com/facebookresearch/fairscale.git
!npm install -g localtunnel

# Clone repositories
!git clone https://github.com/ultralytics/yolov5
!git clone https://github.com/facebookresearch/slowfast
!git clone https://github.com/facebookresearch/pytorchvideo

# Set up YOLOv5
%cd yolov5
!pip install -qr requirements.txt comet_ml
%cd ..

# Set up SlowFast
%cd slowfast
!python setup.py build develop
%cd ..

# Set up PyTorchVideo
%cd pytorchvideo
!pip install -e .
%cd ..

# Add SlowFast to PYTHONPATH
import os
os.environ['PYTHONPATH'] += ":/content/slowfast"

# Download Kinetics-400 labels
!wget -O /content/label_map.txt https://raw.githubusercontent.com/google-deepmind/kinetics-i3d/master/data/label_map.txt

# Create app.py
app_code = """
import streamlit as st
import os
import shutil
import subprocess
import torch
from pytorchvideo.models.hub import slowfast_r50
from pytorchvideo.models.hub import slowfast_r101
import torchvision.transforms as transforms
from pytorchvideo.transforms import ShortSideScale, UniformTemporalSubsample
from pytorchvideo.data.encoded_video import EncodedVideo
import cv2
import time
import io
from PIL import Image

# Ensure detect.py is present
assert os.path.exists('/content/yolov5/detect.py'), 'detect.py not found. Ensure it is in the yolov5 directory.'

@st.cache_data
def process_video_yolov5(video_path):
    output_dir = '/content/runs/detect/exp'
    video_output = os.path.join(output_dir, os.path.basename(video_path))
    crops_dir = os.path.join(output_dir, 'crops', 'cell phone')
    if os.path.exists(output_dir):
        shutil.rmtree(output_dir)  # Clear previous runs
    print(f'Running detect.py on {video_path}')
    command = ['python', '/content/yolov5/detect.py', '--weights', '/content/yolov5/yolov5s.pt', '--source', video_path, '--classes', '67', '--conf', '0.25', '--save-txt', '--save-conf', '--project', '/content/runs/detect', '--exist-ok', '--save-crop']
    result = subprocess.run(command, capture_output=True, text=True)
    print('detect.py has finished running')
    print(result.stdout)
    print(result.stderr)

    # Add a delay to ensure the file is fully written
    time.sleep(5)

    # Gather detected frames
    detected_frames = []
    if os.path.exists(crops_dir):
        for file in os.listdir(crops_dir):
            if file.endswith(('.jpg', '.jpeg', '.png')):
                img_path = os.path.join(crops_dir, file)
                img = Image.open(img_path)
                detected_frames.append(img)

    return video_output, detected_frames

def create_collage(frames, cols=5):
    if len(frames) == 0:
        return None

    # Determine the size of the collage
    frame_width, frame_height = frames[0].size
    rows = (len(frames) + cols - 1) // cols

    collage_width = cols * frame_width
    collage_height = rows * frame_height

    # Create a blank collage
    collage = Image.new('RGB', (collage_width, collage_height))

    # Fill the collage with frames
    for idx, frame in enumerate(frames):
        row = idx // cols
        col = idx % cols
        collage.paste(frame, (col * frame_width, row * frame_height))

    return collage

@st.cache_data
def process_video_pytorchvideo(video_path):
    # Load the pre-trained SlowFast model
    model = slowfast_r101(pretrained="kinetics_600")
    model = model.eval()

    # Load the video using PyTorchVideo's EncodedVideo class
    video = EncodedVideo.from_path(video_path)

    # Select the time range to sample from the video
    start_time = 0
    end_time = min(video.duration, 15)  # Use the shorter of 15 seconds or video duration

    # Sample frames uniformly
    video_data = video.get_clip(start_sec=start_time, end_sec=end_time)
    frames = video_data["video"]

    # Define the transforms to apply to each frame
    transform = transforms.Compose([
        UniformTemporalSubsample(32),
        ShortSideScale(256),
        transforms.CenterCrop(224),
        transforms.Lambda(lambda x: x / 255.0),
    ])

    # Apply the initial transforms
    frames = transform(frames)

    # Normalize the RGB channels
    normalize = transforms.Normalize(mean=[0.45, 0.45, 0.45], std=[0.225, 0.225, 0.225])
    frames = frames.permute(1, 0, 2, 3)
    frames = normalize(frames)
    frames = frames.permute(1, 0, 2, 3)

    # Prepare the input format for SlowFast
    fast_pathway = frames
    slow_pathway = torch.index_select(
        frames,
        1,
        torch.linspace(0, frames.shape[1] - 1, frames.shape[1] // 4).long(),
    )

    # Combine pathways into a list and add a batch dimension
    inputs = [slow_pathway.unsqueeze(0), fast_pathway.unsqueeze(0)]

    # Run inference
    with torch.no_grad():
        preds = model(inputs)

    # Load Kinetics-400 class labels
    kinetics_labels = []
    with open('/content/label_map.txt', 'r') as f:
        kinetics_labels = [line.strip() for line in f if line.strip()]

    # Get top 5 predictions
    top5_pred = torch.topk(preds, k=5)
    top5_actions = []
    for i in range(5):
        if i < len(kinetics_labels):
            action = kinetics_labels[top5_pred.indices[0][i]]
            confidence = top5_pred.values[0][i].item()
            top5_actions.append((action, confidence))
        else:
            break

    return top5_actions

# Initialize session state to store video results
if 'video_results' not in st.session_state:
    st.session_state['video_results'] = []

# Streamlit app logic
st.title('YOLOv5 Cell Phone Detection and PyTorchVideo Action Recognition')

# Allow multiple file uploads
uploaded_files = st.file_uploader('Upload video files', type=['mp4', 'avi', 'mov'], accept_multiple_files=True)

if uploaded_files:
    for uploaded_file in uploaded_files:
        video_path = f'/content/uploadedvideo{uploaded_file.name}'
        with open(video_path, "wb") as f:
            f.write(uploaded_file.getbuffer())

        st.video(uploaded_file)

        # YOLOv5 Inference
        st.subheader(f'YOLOv5 Cell Phone Detection for {uploaded_file.name}')
        yolov5_output, detected_frames = process_video_yolov5(video_path)

        if detected_frames:
            st.write(f'True Positive: {len(detected_frames)} frames detected with cell phones')
            collage_image = create_collage(detected_frames)
            if collage_image:
                st.image(collage_image, caption='Collage of detected frames')
        else:
            st.write('False Positive: No frames with cell phones detected')

        st.video(yolov5_output)

        # Provide a download button for the processed video
        with open(yolov5_output, "rb") as file:
            video_bytes = file.read()
        st.write("You can download the processed video:")
        st.download_button(
            label="Download processed video",
            data=video_bytes,
            file_name=f"processedvideo{uploaded_file.name}",
            mime="video/mp4"
        )

        # SlowFast Inference
        st.subheader(f'PyTorchVideo Action Recognition for {uploaded_file.name}')
        slowfast_output = process_video_pytorchvideo(video_path)

        st.write('Top 5 predicted actions:')
        for i, (action, confidence) in enumerate(slowfast_output, 1):
            st.write(f"{i}. {action}: {confidence:.3f}")

        # Store results for later display
        st.session_state['video_results'].append({
            'file_name': uploaded_file.name,
            'yolov5_output': yolov5_output,
            'slowfast_output': slowfast_output
        })

# Display results for all uploaded videos
if st.session_state.get('video_results'):
    st.write("### Results for all uploaded videos:")
    for result in st.session_state['video_results']:
        st.write(f"#### {result['file_name']}")
        st.video(result['yolov5_output'])

        st.write('Top 5 predicted actions:')
        for i, (action, confidence) in enumerate(result['slowfast_output'], 1):
            st.write(f"{i}. {action}: {confidence:.3f}")

# Add a button to upload another video
if st.button("Add Another Video"):
    st.session_state['upload_interface'] = True

# Conditionally render the upload interface for additional video uploads
if st.session_state.get('upload_interface', True):
    uploaded_file = st.file_uploader('Upload another video file', type=['mp4', 'avi', 'mov'])

    if uploaded_file is not None:
        video_path = f'/content/uploadedvideo{uploaded_file.name}'
        with open(video_path, "wb") as f:
            f.write(uploaded_file.getbuffer())

        st.video(uploaded_file)

        # YOLOv5 Inference
        st.subheader(f'YOLOv5 Cell Phone Detection for {uploaded_file.name}')
        yolov5_output, detected_frames = process_video_yolov5(video_path)

        if detected_frames:
            st.write(f'True Positive: {len(detected_frames)} frames detected with cell phones')
            collage_image = create_collage(detected_frames)
            if collage_image:
                st.image(collage_image, caption='Collage of detected frames')
        else:
            st.write('False Positive: No frames with cell phones detected')

        st.video(yolov5_output)

        # Provide a download button for the processed video
        with open(yolov5_output, "rb") as file:
            video_bytes = file.read()
        st.write("You can download the processed video:")
        st.download_button(
            label="Download processed video",
            data=video_bytes,
            file_name=f"processedvideo{uploaded_file.name}",
            mime="video/mp4"
        )

        # SlowFast Inference
        st.subheader(f'PyTorchVideo Action Recognition for {uploaded_file.name}')
        slowfast_output = process_video_pytorchvideo(video_path)

        st.write('Top 5 predicted actions:')
        for i, (action, confidence) in enumerate(slowfast_output, 1):
            st.write(f"{i}. {action}: {confidence:.3f}")

        # Store the results for later display
        st.session_state['video_results'].append({
            'file_name': uploaded_file.name,
            'yolov5_output': yolov5_output,
            'slowfast_output': slowfast_output
        })

        # Reset the upload interface state to allow another video to be added
        st.session_state['upload_interface'] = False

    # Display the results for all videos after each upload
    if st.session_state.get('video_results'):
        st.write("### Results for all uploaded videos:")
        for result in st.session_state['video_results']:
            st.write(f"#### {result['file_name']}")
            st.video(result['yolov5_output'])

            st.write('Top 5 predicted actions:')
            for i, (action, confidence) in enumerate(result['slowfast_output'], 1):
                st.write(f"{i}. {action}: {confidence:.3f}")
"""

with open("app.py", "w") as file:
    file.write(app_code)
print("app.py file has been written.")

# Start the Streamlit app in the background
import subprocess
streamlit_process = subprocess.Popen(['streamlit', 'run', 'app.py'])

# Start LocalTunnel
import time
import requests

time.sleep(5)
localtunnel_process = subprocess.Popen(['lt', '--port', '8501'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

# Capture the LocalTunnel URL
public_url = None
for line in iter(localtunnel_process.stdout.readline, b''):
    decoded_line = line.decode().strip()
    print(decoded_line)
    if 'your url is:' in decoded_line:
        public_url = decoded_line.split(' ')[-1]
        break

if public_url:
    print(f'Streamlit app running at: {public_url}')
    headers = {
        'User-Agent': 'Mozilla/5.0',
        'bypass-tunnel-reminder': '1'
    }
    response = requests.get(public_url, headers=headers)
    if response.status_code == 200:
        print(f'Access the Streamlit app at: {public_url}')
    else:
        print('Failed to bypass the password screen.')
else:
    print('Failed to create LocalTunnel.')

  Cloning https://github.com/facebookresearch/fvcore.git to /tmp/pip-req-build-p1cvo0m0
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/fvcore.git /tmp/pip-req-build-p1cvo0m0
  Resolved https://github.com/facebookresearch/fvcore.git to commit f3b07ea37daff5b774702708134f7b702e2dc83e
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/facebookresearch/fairscale.git to /tmp/pip-req-build-kx2kdlvq
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/fairscale.git /tmp/pip-req-build-kx2kdlvq
  Resolved https://github.com/facebookresearch/fairscale.git to commit 5f484b3545f27eddb19d970fbe1d361b9c5f2b07
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.7/132.7 kB 11.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!wget -q -O - https://loca.lt/mytunnelpassword

34.126.68.182

In [ ]:
# Install required packages
!pip install streamlit opencv-python-headless pyngrok pytorchvideo torchvision iopath torch git+https://github.com/facebookresearch/fvcore.git git+https://github.com/facebookresearch/fairscale.git
!npm install -g localtunnel

# Clone repositories
!git clone https://github.com/ultralytics/yolov5
!git clone https://github.com/facebookresearch/slowfast
!git clone https://github.com/facebookresearch/pytorchvideo

# Set up YOLOv5
%cd yolov5
!pip install -qr requirements.txt comet_ml
%cd ..

# Set up SlowFast
%cd slowfast
!python setup.py build develop
%cd ..

# Set up PyTorchVideo
%cd pytorchvideo
!pip install -e .
%cd ..

# Add SlowFast to PYTHONPATH
import os
os.environ['PYTHONPATH'] += ":/content/slowfast"

# Download Kinetics-400 labels
!wget -O /content/label_map.txt https://raw.githubusercontent.com/google-deepmind/kinetics-i3d/master/data/label_map.txt

# Create app.py
app_code = """
import streamlit as st
import os
import shutil
import subprocess
import torch
from pytorchvideo.models.hub import slowfast_r50
from pytorchvideo.models.hub import slowfast_r101
import torchvision.transforms as transforms
from pytorchvideo.transforms import ShortSideScale, UniformTemporalSubsample
from pytorchvideo.data.encoded_video import EncodedVideo
import cv2
import time
import io
from PIL import Image

# Ensure detect.py is present
assert os.path.exists('/content/yolov5/detect.py'), 'detect.py not found. Ensure it is in the yolov5 directory.'

@st.cache_data
def process_video_yolov5(video_path):
    output_dir = '/content/runs/detect/exp'
    crops_dir = os.path.join(output_dir, 'crops', 'cell phone')
    if os.path.exists(output_dir):
        shutil.rmtree(output_dir)  # Clear previous runs
    print(f'Running detect.py on {video_path}')
    command = ['python', '/content/yolov5/detect.py', '--weights', '/content/yolov5/yolov5s.pt', '--source', video_path, '--classes', '67', '--conf', '0.25', '--save-txt', '--save-conf', '--project', '/content/runs/detect', '--exist-ok', '--save-crop']
    result = subprocess.run(command, capture_output=True, text=True)
    print('detect.py has finished running')
    print(result.stdout)
    print(result.stderr)

    # Add a delay to ensure the file is fully written
    time.sleep(5)

    # Gather detected frames
    detected_frames = []
    if os.path.exists(crops_dir):
        for file in os.listdir(crops_dir):
            if file.endswith(('.jpg', '.jpeg', '.png')):
                img_path = os.path.join(crops_dir, file)
                img = Image.open(img_path)
                detected_frames.append(img)

    return detected_frames

def create_collage(frames, cols=5):
    if len(frames) == 0:
        return None

    # Determine the size of the collage
    frame_width, frame_height = frames[0].size
    rows = (len(frames) + cols - 1) // cols

    collage_width = cols * frame_width
    collage_height = rows * frame_height

    # Create a blank collage
    collage = Image.new('RGB', (collage_width, collage_height))

    # Fill the collage with frames
    for idx, frame in enumerate(frames):
        row = idx // cols
        col = idx % cols
        collage.paste(frame, (col * frame_width, row * frame_height))

    return collage

@st.cache_data
def process_video_pytorchvideo(video_path):
    # Load the pre-trained SlowFast model
    model = slowfast_r101(pretrained="kinetics_600")
    model = model.eval()

    # Load the video using PyTorchVideo's EncodedVideo class
    video = EncodedVideo.from_path(video_path)

    # Select the time range to sample from the video
    start_time = 0
    end_time = min(video.duration, 15)  # Use the shorter of 15 seconds or video duration

    # Sample frames uniformly
    video_data = video.get_clip(start_sec=start_time, end_sec=end_time)
    frames = video_data["video"]

    # Define the transforms to apply to each frame
    transform = transforms.Compose([
        UniformTemporalSubsample(32),
        ShortSideScale(256),
        transforms.CenterCrop(224),
        transforms.Lambda(lambda x: x / 255.0),
    ])

    # Apply the initial transforms
    frames = transform(frames)

    # Normalize the RGB channels
    normalize = transforms.Normalize(mean=[0.45, 0.45, 0.45], std=[0.225, 0.225, 0.225])
    frames = frames.permute(1, 0, 2, 3)
    frames = normalize(frames)
    frames = frames.permute(1, 0, 2, 3)

    # Prepare the input format for SlowFast
    fast_pathway = frames
    slow_pathway = torch.index_select(
        frames,
        1,
        torch.linspace(0, frames.shape[1] - 1, frames.shape[1] // 4).long(),
    )

    # Combine pathways into a list and add a batch dimension
    inputs = [slow_pathway.unsqueeze(0), fast_pathway.unsqueeze(0)]

    # Run inference
    with torch.no_grad():
        preds = model(inputs)

    # Load Kinetics-400 class labels
    kinetics_labels = []
    with open('/content/label_map.txt', 'r') as f:
        kinetics_labels = [line.strip() for line in f if line.strip()]

    # Get top 5 predictions
    top5_pred = torch.topk(preds, k=5)
    top5_actions = []
    for i in range(5):
        if i < len(kinetics_labels):
            action = kinetics_labels[top5_pred.indices[0][i]]
            confidence = top5_pred.values[0][i].item()
            top5_actions.append((action, confidence))
        else:
            break

    return top5_actions

def process_video(uploaded_file):
    video_path = f'/content/uploadedvideo{uploaded_file.name}'
    with open(video_path, "wb") as f:
        f.write(uploaded_file.getbuffer())

    st.video(uploaded_file)

    # YOLOv5 Inference
    st.subheader(f'YOLOv5 Cell Phone Detection for {uploaded_file.name}')
    detected_frames = process_video_yolov5(video_path)

    if detected_frames:
        st.write(f'True Positive: {len(detected_frames)} frames detected with cell phones')
        collage_image = create_collage(detected_frames)
        if collage_image:
            st.image(collage_image, caption='Collage of detected frames')
    else:
        st.write('False Positive: No frames with cell phones detected')

    # SlowFast Inference
    st.subheader(f'PyTorchVideo Action Recognition for {uploaded_file.name}')
    slowfast_output = process_video_pytorchvideo(video_path)

    st.write('Top 5 predicted actions:')
    for i, (action, confidence) in enumerate(slowfast_output, 1):
        st.write(f"{i}. {action}: {confidence:.3f}")

# Streamlit app logic
st.title('YOLOv5 Cell Phone Detection and PyTorchVideo Action Recognition')

# Initialize session state
if 'show_upload_interface' not in st.session_state:
    st.session_state['show_upload_interface'] = False

# Allow multiple file uploads
uploaded_files = st.file_uploader('Upload video files', type=['mp4', 'avi', 'mov'], accept_multiple_files=True)

if uploaded_files:
    for uploaded_file in uploaded_files:
        process_video(uploaded_file)

# Add a button to upload another video
if st.button("Add Another Video"):
    st.session_state['show_upload_interface'] = True

# Conditionally render the upload interface for additional video uploads
if st.session_state['show_upload_interface']:
    uploaded_file = st.file_uploader('Upload another video file', type=['mp4', 'avi', 'mov'], key='additional_upload')

    if uploaded_file is not None:
        process_video(uploaded_file)
        st.session_state['show_upload_interface'] = False
"""

with open("app.py", "w") as file:
    file.write(app_code)
print("app.py file has been written.")

# Start the Streamlit app in the background
import subprocess
streamlit_process = subprocess.Popen(['streamlit', 'run', 'app.py'])

# Start LocalTunnel
import time
import requests

time.sleep(5)
localtunnel_process = subprocess.Popen(['lt', '--port', '8501'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

# Capture the LocalTunnel URL
public_url = None
for line in iter(localtunnel_process.stdout.readline, b''):
    decoded_line = line.decode().strip()
    print(decoded_line)
    if 'your url is:' in decoded_line:
        public_url = decoded_line.split(' ')[-1]
        break

if public_url:
    print(f'Streamlit app running at: {public_url}')
    headers = {
        'User-Agent': 'Mozilla/5.0',
        'bypass-tunnel-reminder': '1'
    }
    response = requests.get(public_url, headers=headers)
    if response.status_code == 200:
        print(f'Access the Streamlit app at: {public_url}')
    else:
        print('Failed to bypass the password screen.')
else:
    print('Failed to create LocalTunnel.')

  Cloning https://github.com/facebookresearch/fvcore.git to /tmp/pip-req-build-o3wecv00
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/fvcore.git /tmp/pip-req-build-o3wecv00
  Resolved https://github.com/facebookresearch/fvcore.git to commit f3b07ea37daff5b774702708134f7b702e2dc83e
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/facebookresearch/fairscale.git to /tmp/pip-req-build-thg15_2_
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/fairscale.git /tmp/pip-req-build-thg15_2_
  Resolved https://github.com/facebookresearch/fairscale.git to commit 5f484b3545f27eddb19d970fbe1d361b9c5f2b07
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.7/132.7 kB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!wget -q -O - https://loca.lt/mytunnelpassword

34.125.189.115

In [ ]:
# Install required packages
!pip install streamlit opencv-python-headless pyngrok pytorchvideo torchvision iopath torch git+https://github.com/facebookresearch/fvcore.git git+https://github.com/facebookresearch/fairscale.git
!npm install -g localtunnel

# Clone repositories
!git clone https://github.com/ultralytics/yolov5
!git clone https://github.com/facebookresearch/slowfast
!git clone https://github.com/facebookresearch/pytorchvideo

# Set up YOLOv5
%cd yolov5
!pip install -qr requirements.txt comet_ml
%cd ..

# Set up SlowFast
%cd slowfast
!python setup.py build develop
%cd ..

# Set up PyTorchVideo
%cd pytorchvideo
!pip install -e .
%cd ..

# Add SlowFast to PYTHONPATH
import os
os.environ['PYTHONPATH'] += ":/content/slowfast"

# Download Kinetics-400 labels
!wget -O /content/label_map.txt https://raw.githubusercontent.com/google-deepmind/kinetics-i3d/master/data/label_map.txt

# Create app.py
app_code = """
import streamlit as st
import os
import shutil
import subprocess
import torch
from pytorchvideo.models.hub import slowfast_r50
from pytorchvideo.models.hub import slowfast_r101
import torchvision.transforms as transforms
from pytorchvideo.transforms import ShortSideScale, UniformTemporalSubsample
from pytorchvideo.data.encoded_video import EncodedVideo
import cv2
import time
import io
from PIL import Image

# Ensure detect.py is present
assert os.path.exists('/content/yolov5/detect.py'), 'detect.py not found. Ensure it is in the yolov5 directory.'

@st.cache_data
def process_video_yolov5(video_path):
    output_dir = '/content/runs/detect/exp'
    video_output = os.path.join(output_dir, os.path.basename(video_path))
    crops_dir = os.path.join(output_dir, 'crops', 'cell phone')
    if os.path.exists(output_dir):
        shutil.rmtree(output_dir)  # Clear previous runs
    print(f'Running detect.py on {video_path}')
    command = ['python', '/content/yolov5/detect.py', '--weights', '/content/yolov5/yolov5s.pt', '--source', video_path, '--classes', '67', '--conf', '0.25', '--save-txt', '--save-conf', '--project', '/content/runs/detect', '--exist-ok', '--save-crop']
    result = subprocess.run(command, capture_output=True, text=True)
    print('detect.py has finished running')
    print(result.stdout)
    print(result.stderr)

    # Add a delay to ensure the file is fully written
    time.sleep(5)

    # Gather detected frames
    detected_frames = []
    if os.path.exists(crops_dir):
        for file in os.listdir(crops_dir):
            if file.endswith(('.jpg', '.jpeg', '.png')):
                img_path = os.path.join(crops_dir, file)
                img = Image.open(img_path)
                detected_frames.append(img)

    return video_output, detected_frames

def create_collage(frames, cols=5):
    if len(frames) == 0:
        return None

    # Determine the size of the collage
    frame_width, frame_height = frames[0].size
    rows = (len(frames) + cols - 1) // cols

    collage_width = cols * frame_width
    collage_height = rows * frame_height

    # Create a blank collage
    collage = Image.new('RGB', (collage_width, collage_height))

    # Fill the collage with frames
    for idx, frame in enumerate(frames):
        row = idx // cols
        col = idx % cols
        collage.paste(frame, (col * frame_width, row * frame_height))

    return collage

@st.cache_data
def process_video_pytorchvideo(video_path):
    # Load the pre-trained SlowFast model
    model = slowfast_r101(pretrained="kinetics_600")
    model = model.eval()

    # Load the video using PyTorchVideo's EncodedVideo class
    video = EncodedVideo.from_path(video_path)

    # Select the time range to sample from the video
    start_time = 0
    end_time = min(video.duration, 15)  # Use the shorter of 15 seconds or video duration

    # Sample frames uniformly
    video_data = video.get_clip(start_sec=start_time, end_sec=end_time)
    frames = video_data["video"]

    # Define the transforms to apply to each frame
    transform = transforms.Compose([
        UniformTemporalSubsample(32),
        ShortSideScale(256),
        transforms.CenterCrop(224),
        transforms.Lambda(lambda x: x / 255.0),
    ])

    # Apply the initial transforms
    frames = transform(frames)

    # Normalize the RGB channels
    normalize = transforms.Normalize(mean=[0.45, 0.45, 0.45], std=[0.225, 0.225, 0.225])
    frames = frames.permute(1, 0, 2, 3)
    frames = normalize(frames)
    frames = frames.permute(1, 0, 2, 3)

    # Prepare the input format for SlowFast
    fast_pathway = frames
    slow_pathway = torch.index_select(
        frames,
        1,
        torch.linspace(0, frames.shape[1] - 1, frames.shape[1] // 4).long(),
    )

    # Combine pathways into a list and add a batch dimension
    inputs = [slow_pathway.unsqueeze(0), fast_pathway.unsqueeze(0)]

    # Run inference
    with torch.no_grad():
        preds = model(inputs)

    # Load Kinetics-400 class labels
    kinetics_labels = []
    with open('/content/label_map.txt', 'r') as f:
        kinetics_labels = [line.strip() for line in f if line.strip()]

    # Get top 5 predictions
    top5_pred = torch.topk(preds, k=5)
    top5_actions = []
    for i in range(5):
        if i < len(kinetics_labels):
            action = kinetics_labels[top5_pred.indices[0][i]]
            confidence = top5_pred.values[0][i].item()
            top5_actions.append((action, confidence))
        else:
            break

    return top5_actions

def process_video(uploaded_file):
    video_path = f'/content/uploadedvideo{uploaded_file.name}'
    with open(video_path, "wb") as f:
        f.write(uploaded_file.getbuffer())

    # Display the original uploaded video
    st.video(uploaded_file)

    # YOLOv5 Inference
    st.subheader(f'YOLOv5 Cell Phone Detection for {uploaded_file.name}')
    yolov5_output, detected_frames = process_video_yolov5(video_path)

    if detected_frames:
        st.write(f'True Positive: {len(detected_frames)} frames detected with cell phones')
        collage_image = create_collage(detected_frames)
        if collage_image:
            st.image(collage_image, caption='Collage of detected frames')
    else:
        st.write('False Positive: No frames with cell phones detected')

    # Provide a download button for the processed video
    with open(yolov5_output, "rb") as file:
        video_bytes = file.read()
    st.write("You can download the processed video:")
    st.download_button(
        label="Download processed video",
        data=video_bytes,
        file_name=f"processedvideo{uploaded_file.name}",
        mime="video/mp4"
    )

    # SlowFast Inference
    st.subheader(f'PyTorchVideo Action Recognition for {uploaded_file.name}')
    slowfast_output = process_video_pytorchvideo(video_path)

    st.write('Top 5 predicted actions:')
    for i, (action, confidence) in enumerate(slowfast_output, 1):
        st.write(f"{i}. {action}: {confidence:.3f}")

# Streamlit app logic
st.title('YOLOv5 Cell Phone Detection and PyTorchVideo Action Recognition')

# Allow multiple file uploads
uploaded_files = st.file_uploader('Upload video files', type=['mp4', 'avi', 'mov'], accept_multiple_files=True)

if uploaded_files:
    for uploaded_file in uploaded_files:
        process_video(uploaded_file)

# Always show the "Add Another Video" button
if st.button("Add Another Video", key="add_another"):
    uploaded_file = st.file_uploader('Upload another video file', type=['mp4', 'avi', 'mov'], key='additional_upload')
    if uploaded_file:
        process_video(uploaded_file)

"""
with open("app.py", "w") as file:
    file.write(app_code)
print("app.py file has been written.")

# Start the Streamlit app in the background
import subprocess
streamlit_process = subprocess.Popen(['streamlit', 'run', 'app.py'])

# Start LocalTunnel
import time
import requests

time.sleep(5)
localtunnel_process = subprocess.Popen(['lt', '--port', '8501'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

# Capture the LocalTunnel URL
public_url = None
for line in iter(localtunnel_process.stdout.readline, b''):
    decoded_line = line.decode().strip()
    print(decoded_line)
    if 'your url is:' in decoded_line:
        public_url = decoded_line.split(' ')[-1]
        break

if public_url:
    print(f'Streamlit app running at: {public_url}')
    headers = {
        'User-Agent': 'Mozilla/5.0',
        'bypass-tunnel-reminder': '1'
    }
    response = requests.get(public_url, headers=headers)
    if response.status_code == 200:
        print(f'Access the Streamlit app at: {public_url}')
    else:
        print('Failed to bypass the password screen.')
else:
    print('Failed to create LocalTunnel.')

  Cloning https://github.com/facebookresearch/fvcore.git to /tmp/pip-req-build-22a2spgp
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/fvcore.git /tmp/pip-req-build-22a2spgp
  Resolved https://github.com/facebookresearch/fvcore.git to commit f3b07ea37daff5b774702708134f7b702e2dc83e
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/facebookresearch/fairscale.git to /tmp/pip-req-build-dby78oo2
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/fairscale.git /tmp/pip-req-build-dby78oo2
  Resolved https://github.com/facebookresearch/fairscale.git to commit 5f484b3545f27eddb19d970fbe1d361b9c5f2b07
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.7/132.7 kB 5.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!wget -q -O - https://loca.lt/mytunnelpassword

34.145.39.206

In [ ]:
# Install required packages
!pip install streamlit opencv-python-headless pyngrok pytorchvideo torchvision iopath torch git+https://github.com/facebookresearch/fvcore.git git+https://github.com/facebookresearch/fairscale.git
!npm install -g localtunnel

# Clone repositories
!git clone https://github.com/ultralytics/yolov5
!git clone https://github.com/facebookresearch/slowfast
!git clone https://github.com/facebookresearch/pytorchvideo

# Set up YOLOv5
%cd yolov5
!pip install -qr requirements.txt comet_ml
%cd ..

# Set up SlowFast
%cd slowfast
!python setup.py build develop
%cd ..

# Set up PyTorchVideo
%cd pytorchvideo
!pip install -e .
%cd ..

# Add SlowFast to PYTHONPATH
import os
os.environ['PYTHONPATH'] += ":/content/slowfast"

# Download Kinetics-400 labels
!wget -O /content/label_map.txt https://raw.githubusercontent.com/google-deepmind/kinetics-i3d/master/data/label_map.txt

# Create app.py
app_code = """
import streamlit as st
import os
import shutil
import subprocess
import torch
from pytorchvideo.models.hub import slowfast_r50
from pytorchvideo.models.hub import slowfast_r101
import torchvision.transforms as transforms
from pytorchvideo.transforms import ShortSideScale, UniformTemporalSubsample
from pytorchvideo.data.encoded_video import EncodedVideo
import cv2
import time
import io
from PIL import Image

# Ensure detect.py is present
assert os.path.exists('/content/yolov5/detect.py'), 'detect.py not found. Ensure it is in the yolov5 directory.'

@st.cache_data
def process_video_yolov5(video_path):
    output_dir = '/content/runs/detect/exp'
    video_output = os.path.join(output_dir, os.path.basename(video_path))
    crops_dir = os.path.join(output_dir, 'crops', 'cell phone')
    if os.path.exists(output_dir):
        shutil.rmtree(output_dir)  # Clear previous runs
    print(f'Running detect.py on {video_path}')
    command = ['python', '/content/yolov5/detect.py', '--weights', '/content/yolov5/yolov5s.pt', '--source', video_path, '--classes', '67', '--conf', '0.25', '--save-txt', '--save-conf', '--project', '/content/runs/detect', '--exist-ok', '--save-crop']
    result = subprocess.run(command, capture_output=True, text=True)
    print('detect.py has finished running')
    print(result.stdout)
    print(result.stderr)

    # Add a delay to ensure the file is fully written
    time.sleep(5)

    # Gather detected frames
    detected_frames = []
    if os.path.exists(crops_dir):
        for file in os.listdir(crops_dir):
            if file.endswith(('.jpg', '.jpeg', '.png')):
                img_path = os.path.join(crops_dir, file)
                img = Image.open(img_path)
                detected_frames.append(img)

    return video_output, detected_frames

def create_collage(frames, cols=5):
    if len(frames) == 0:
        return None

    # Determine the size of the collage
    frame_width, frame_height = frames[0].size
    rows = (len(frames) + cols - 1) // cols

    collage_width = cols * frame_width
    collage_height = rows * frame_height

    # Create a blank collage
    collage = Image.new('RGB', (collage_width, collage_height))

    # Fill the collage with frames
    for idx, frame in enumerate(frames):
        row = idx // cols
        col = idx % cols
        collage.paste(frame, (col * frame_width, row * frame_height))

    return collage

@st.cache_data
def process_video_pytorchvideo(video_path):
    # Load the pre-trained SlowFast model
    model = slowfast_r101(pretrained="kinetics_600")
    model = model.eval()

    # Load the video using PyTorchVideo's EncodedVideo class
    video = EncodedVideo.from_path(video_path)

    # Select the time range to sample from the video
    start_time = 0
    end_time = min(video.duration, 15)  # Use the shorter of 15 seconds or video duration

    # Sample frames uniformly
    video_data = video.get_clip(start_sec=start_time, end_sec=end_time)
    frames = video_data["video"]

    # Define the transforms to apply to each frame
    transform = transforms.Compose([
        UniformTemporalSubsample(32),
        ShortSideScale(256),
        transforms.CenterCrop(224),
        transforms.Lambda(lambda x: x / 255.0),
    ])

    # Apply the initial transforms
    frames = transform(frames)

    # Normalize the RGB channels
    normalize = transforms.Normalize(mean=[0.45, 0.45, 0.45], std=[0.225, 0.225, 0.225])
    frames = frames.permute(1, 0, 2, 3)
    frames = normalize(frames)
    frames = frames.permute(1, 0, 2, 3)

    # Prepare the input format for SlowFast
    fast_pathway = frames
    slow_pathway = torch.index_select(
        frames,
        1,
        torch.linspace(0, frames.shape[1] - 1, frames.shape[1] // 4).long(),
    )

    # Combine pathways into a list and add a batch dimension
    inputs = [slow_pathway.unsqueeze(0), fast_pathway.unsqueeze(0)]

    # Run inference
    with torch.no_grad():
        preds = model(inputs)

    # Load Kinetics-400 class labels
    kinetics_labels = []
    with open('/content/label_map.txt', 'r') as f:
        kinetics_labels = [line.strip() for line in f if line.strip()]

    # Get top 5 predictions
    top5_pred = torch.topk(preds, k=5)
    top5_actions = []
    for i in range(5):
        if i < len(kinetics_labels):
            action = kinetics_labels[top5_pred.indices[0][i]]
            confidence = top5_pred.values[0][i].item()
            top5_actions.append((action, confidence))
        else:
            break

    return top5_actions

def process_video(uploaded_file):
    if uploaded_file is not None:
        video_path = f'/content/uploadedvideo{uploaded_file.name}'
        with open(video_path, "wb") as f:
            f.write(uploaded_file.getbuffer())

        # Display the original uploaded video
        st.video(uploaded_file)

        # YOLOv5 Inference
        st.subheader(f'YOLOv5 Cell Phone Detection for {uploaded_file.name}')
        yolov5_output, detected_frames = process_video_yolov5(video_path)

        if detected_frames:
            st.write(f'True Positive: {len(detected_frames)} frames detected with cell phones')
            collage_image = create_collage(detected_frames)
            if collage_image:
                st.image(collage_image, caption='Collage of detected frames')
        else:
            st.write('False Positive: No frames with cell phones detected')

        # Provide a download button for the processed video
        with open(yolov5_output, "rb") as file:
            video_bytes = file.read()
        st.write("You can download the processed video:")
        st.download_button(
            label="Download processed video",
            data=video_bytes,
            file_name=f"processedvideo{uploaded_file.name}",
            mime="video/mp4"
        )

        # SlowFast Inference
        st.subheader(f'PyTorchVideo Action Recognition for {uploaded_file.name}')
        slowfast_output = process_video_pytorchvideo(video_path)

        st.write('Top 5 predicted actions:')
        for i, (action, confidence) in enumerate(slowfast_output, 1):
            st.write(f"{i}. {action}: {confidence:.3f}")

# Streamlit app logic
st.title('YOLOv5 Cell Phone Detection and PyTorchVideo Action Recognition')

# Allow multiple file uploads
uploaded_files = st.file_uploader('Upload video files', type=['mp4', 'avi', 'mov'], accept_multiple_files=True)

if uploaded_files:
    for uploaded_file in uploaded_files:
        process_video(uploaded_file)

# Add Another Video functionality
if st.button("Add Another Video"):
    additional_file = st.file_uploader('Upload another video file', type=['mp4', 'avi', 'mov'], key='additional_upload')
    if additional_file:
        process_video(additional_file)

"""
with open("app.py", "w") as file:
    file.write(app_code)
print("app.py file has been written.")

# Start the Streamlit app in the background
import subprocess
streamlit_process = subprocess.Popen(['streamlit', 'run', 'app.py'])

# Start LocalTunnel
import time
import requests

time.sleep(5)
localtunnel_process = subprocess.Popen(['lt', '--port', '8501'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

# Capture the LocalTunnel URL
public_url = None
for line in iter(localtunnel_process.stdout.readline, b''):
    decoded_line = line.decode().strip()
    print(decoded_line)
    if 'your url is:' in decoded_line:
        public_url = decoded_line.split(' ')[-1]
        break

if public_url:
    print(f'Streamlit app running at: {public_url}')
    headers = {
        'User-Agent': 'Mozilla/5.0',
        'bypass-tunnel-reminder': '1'
    }
    response = requests.get(public_url, headers=headers)
    if response.status_code == 200:
        print(f'Access the Streamlit app at: {public_url}')
    else:
        print('Failed to bypass the password screen.')
else:
    print('Failed to create LocalTunnel.')

  Cloning https://github.com/facebookresearch/fvcore.git to /tmp/pip-req-build-64do1bu1
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/fvcore.git /tmp/pip-req-build-64do1bu1
  Resolved https://github.com/facebookresearch/fvcore.git to commit f3b07ea37daff5b774702708134f7b702e2dc83e
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/facebookresearch/fairscale.git to /tmp/pip-req-build-rh9vfl1i
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/fairscale.git /tmp/pip-req-build-rh9vfl1i
  Resolved https://github.com/facebookresearch/fairscale.git to commit 5f484b3545f27eddb19d970fbe1d361b9c5f2b07
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.7/132.7 kB 5.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!wget -q -O - https://loca.lt/mytunnelpassword

34.16.190.45

In [1]:
# Install required packages
!pip install streamlit opencv-python-headless pyngrok pytorchvideo torchvision iopath torch git+https://github.com/facebookresearch/fvcore.git git+https://github.com/facebookresearch/fairscale.git
!npm install -g localtunnel

# Clone repositories
!git clone https://github.com/ultralytics/yolov5
!git clone https://github.com/facebookresearch/slowfast
!git clone https://github.com/facebookresearch/pytorchvideo

# Set up YOLOv5
%cd yolov5
!pip install -qr requirements.txt comet_ml
%cd ..

# Set up SlowFast
%cd slowfast
!python setup.py build develop
%cd ..

# Set up PyTorchVideo
%cd pytorchvideo
!pip install -e .
%cd ..

# Add SlowFast to PYTHONPATH
import os
os.environ['PYTHONPATH'] += ":/content/slowfast"

# Download Kinetics-400 labels
!wget -O /content/label_map.txt https://raw.githubusercontent.com/google-deepmind/kinetics-i3d/master/data/label_map.txt

# Create app.py
app_code = """
import streamlit as st
import os
import shutil
import subprocess
import torch
from pytorchvideo.models.hub import slowfast_r50
from pytorchvideo.models.hub import slowfast_r101
import torchvision.transforms as transforms
from pytorchvideo.transforms import ShortSideScale, UniformTemporalSubsample
from pytorchvideo.data.encoded_video import EncodedVideo
import cv2
import time
import io
from PIL import Image

# Ensure detect.py is present
assert os.path.exists('/content/yolov5/detect.py'), 'detect.py not found. Ensure it is in the yolov5 directory.'

@st.cache_data
def process_video_yolov5(video_path):
    output_dir = '/content/runs/detect/exp'
    video_output = os.path.join(output_dir, os.path.basename(video_path))
    crops_dir = os.path.join(output_dir, 'crops', 'cell phone')
    if os.path.exists(output_dir):
        shutil.rmtree(output_dir)  # Clear previous runs
    print(f'Running detect.py on {video_path}')
    command = ['python', '/content/yolov5/detect.py', '--weights', '/content/yolov5/yolov5l.pt', '--source', video_path, '--classes', '67', '--conf', '0.25', '--save-txt', '--save-conf', '--project', '/content/runs/detect', '--exist-ok', '--save-crop']
    result = subprocess.run(command, capture_output=True, text=True)
    print('detect.py has finished running')
    print(result.stdout)
    print(result.stderr)

    # Add a delay to ensure the file is fully written
    time.sleep(5)

    # Gather detected frames
    detected_frames = []
    if os.path.exists(crops_dir):
        for file in os.listdir(crops_dir):
            if file.endswith(('.jpg', '.jpeg', '.png')):
                img_path = os.path.join(crops_dir, file)
                img = Image.open(img_path)
                detected_frames.append(img)

    return video_output, detected_frames

def create_collage(frames, cols=5):
    if len(frames) == 0:
        return None

    # Determine the size of the collage
    frame_width, frame_height = frames[0].size
    rows = (len(frames) + cols - 1) // cols

    collage_width = cols * frame_width
    collage_height = rows * frame_height

    # Create a blank collage
    collage = Image.new('RGB', (collage_width, collage_height))

    # Fill the collage with frames
    for idx, frame in enumerate(frames):
        row = idx // cols
        col = idx % cols
        collage.paste(frame, (col * frame_width, row * frame_height))

    return collage

@st.cache_data
def process_video_pytorchvideo(video_path):
    # Load the pre-trained SlowFast model
    model = slowfast_r101(pretrained="kinetics_600")
    model = model.eval()

    # Load the video using PyTorchVideo's EncodedVideo class
    video = EncodedVideo.from_path(video_path)

    # Select the time range to sample from the video
    start_time = 0
    end_time = min(video.duration, 15)  # Use the shorter of 15 seconds or video duration

    # Sample frames uniformly
    video_data = video.get_clip(start_sec=start_time, end_sec=end_time)
    frames = video_data["video"]

    # Define the transforms to apply to each frame
    transform = transforms.Compose([
        UniformTemporalSubsample(32),
        ShortSideScale(256),
        transforms.CenterCrop(224),
        transforms.Lambda(lambda x: x / 255.0),
    ])

    # Apply the initial transforms
    frames = transform(frames)

    # Normalize the RGB channels
    normalize = transforms.Normalize(mean=[0.45, 0.45, 0.45], std=[0.225, 0.225, 0.225])
    frames = frames.permute(1, 0, 2, 3)
    frames = normalize(frames)
    frames = frames.permute(1, 0, 2, 3)

    # Prepare the input format for SlowFast
    fast_pathway = frames
    slow_pathway = torch.index_select(
        frames,
        1,
        torch.linspace(0, frames.shape[1] - 1, frames.shape[1] // 4).long(),
    )

    # Combine pathways into a list and add a batch dimension
    inputs = [slow_pathway.unsqueeze(0), fast_pathway.unsqueeze(0)]

    # Run inference
    with torch.no_grad():
        preds = model(inputs)

    # Load Kinetics-400 class labels
    kinetics_labels = []
    with open('/content/label_map.txt', 'r') as f:
        kinetics_labels = [line.strip() for line in f if line.strip()]

    # Get top 5 predictions
    top5_pred = torch.topk(preds, k=5)
    top5_actions = []
    for i in range(5):
        if i < len(kinetics_labels):
            action = kinetics_labels[top5_pred.indices[0][i]]
            confidence = top5_pred.values[0][i].item()
            top5_actions.append((action, confidence))
        else:
            break

    return top5_actions

def process_video(uploaded_file):
    if uploaded_file is not None:
        video_path = f'/content/uploadedvideo{uploaded_file.name}'
        with open(video_path, "wb") as f:
            f.write(uploaded_file.getbuffer())

        # Display the original uploaded video
        st.video(uploaded_file)

        # YOLOv5 Inference
        st.subheader(f'YOLOv5 Cell Phone Detection for {uploaded_file.name}')
        yolov5_output, detected_frames = process_video_yolov5(video_path)

        if detected_frames:
            st.write(f'True Positive: {len(detected_frames)} frames detected with cell phones')
            collage_image = create_collage(detected_frames)
            if collage_image:
                st.image(collage_image, caption='Collage of detected frames')
        else:
            st.write('False Positive: No frames with cell phones detected')

        # Provide a download button for the processed video
        with open(yolov5_output, "rb") as file:
            video_bytes = file.read()
        st.write("You can download the processed video:")
        st.download_button(
            label="Download processed video",
            data=video_bytes,
            file_name=f"processedvideo{uploaded_file.name}",
            mime="video/mp4"
        )

        # SlowFast Inference
        st.subheader(f'PyTorchVideo Action Recognition for {uploaded_file.name}')
        slowfast_output = process_video_pytorchvideo(video_path)

        st.write('Top 5 predicted actions:')
        for i, (action, confidence) in enumerate(slowfast_output, 1):
            st.write(f"{i}. {action}: {confidence:.3f}")

# Streamlit app logic
st.title('YOLOv5 Cell Phone Detection and PyTorchVideo Action Recognition')

# Allow multiple file uploads
uploaded_files = st.file_uploader('Upload video files', type=['mp4', 'avi', 'mov'], accept_multiple_files=True)

if uploaded_files:
    for uploaded_file in uploaded_files:
        process_video(uploaded_file)

# Add Another Video functionality
if st.button("Add Another Video"):
    additional_file = st.file_uploader('Upload another video file', type=['mp4', 'avi', 'mov'], key='additional_upload')
    if additional_file:
        process_video(additional_file)

"""
with open("app.py", "w") as file:
    file.write(app_code)
print("app.py file has been written.")

# Start the Streamlit app in the background
import subprocess
streamlit_process = subprocess.Popen(['streamlit', 'run', 'app.py'])

# Start LocalTunnel
import time
import requests

time.sleep(5)
localtunnel_process = subprocess.Popen(['lt', '--port', '8501'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

# Capture the LocalTunnel URL
public_url = None
for line in iter(localtunnel_process.stdout.readline, b''):
    decoded_line = line.decode().strip()
    print(decoded_line)
    if 'your url is:' in decoded_line:
        public_url = decoded_line.split(' ')[-1]
        break

if public_url:
    print(f'Streamlit app running at: {public_url}')
    headers = {
        'User-Agent': 'Mozilla/5.0',
        'bypass-tunnel-reminder': '1'
    }
    response = requests.get(public_url, headers=headers)
    if response.status_code == 200:
        print(f'Access the Streamlit app at: {public_url}')
    else:
        print('Failed to bypass the password screen.')
else:
    print('Failed to create LocalTunnel.')

  Cloning https://github.com/facebookresearch/fvcore.git to /tmp/pip-req-build-e5uon4v5
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/fvcore.git /tmp/pip-req-build-e5uon4v5
  Resolved https://github.com/facebookresearch/fvcore.git to commit f3b07ea37daff5b774702708134f7b702e2dc83e
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/facebookresearch/fairscale.git to /tmp/pip-req-build-b15n1z1s
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/fairscale.git /tmp/pip-req-build-b15n1z1s
  Resolved https://github.com/facebookresearch/fairscale.git to commit 5f484b3545f27eddb19d970fbe1d361b9c5f2b07
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.7/132.7 kB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!wget -q -O - https://loca.lt/mytunnelpassword

34.83.195.150

In [1]:
# Install required packages
!pip install streamlit opencv-python-headless pyngrok pytorchvideo torchvision iopath torch git+https://github.com/facebookresearch/fvcore.git git+https://github.com/facebookresearch/fairscale.git
!npm install -g localtunnel

# Clone repositories
!git clone https://github.com/ultralytics/yolov5
!git clone https://github.com/facebookresearch/slowfast
!git clone https://github.com/facebookresearch/pytorchvideo

# Set up YOLOv5
%cd yolov5
!pip install -qr requirements.txt comet_ml
%cd ..

# Set up SlowFast
%cd slowfast
!python setup.py build develop
%cd ..

# Set up PyTorchVideo
%cd pytorchvideo
!pip install -e .
%cd ..

# Add SlowFast to PYTHONPATH
import os
os.environ['PYTHONPATH'] += ":/content/slowfast"

# Download Kinetics-400 labels
!wget -O /content/label_map.txt https://raw.githubusercontent.com/google-deepmind/kinetics-i3d/master/data/label_map.txt

# Create app.py
app_code = """
import streamlit as st
import os
import shutil
import subprocess
import torch
from pytorchvideo.models.hub import slowfast_r50
from pytorchvideo.models.hub import slowfast_r101
import torchvision.transforms as transforms
from pytorchvideo.transforms import ShortSideScale, UniformTemporalSubsample
from pytorchvideo.data.encoded_video import EncodedVideo
import cv2
import time
import io
from PIL import Image

# Ensure detect.py is present
assert os.path.exists('/content/yolov5/detect.py'), 'detect.py not found. Ensure it is in the yolov5 directory.'

@st.cache_data
def process_video_yolov5(video_path):
    output_dir = '/content/runs/detect/exp'
    video_output = os.path.join(output_dir, os.path.basename(video_path))
    crops_dir = os.path.join(output_dir, 'crops', 'cell phone')
    if os.path.exists(output_dir):
        shutil.rmtree(output_dir)  # Clear previous runs
    print(f'Running detect.py on {video_path}')
    command = ['python', '/content/yolov5/detect.py', '--weights', '/content/yolov5/yolov5x.pt', '--source', video_path, '--classes', '67', '--conf', '0.25', '--save-txt', '--save-conf', '--project', '/content/runs/detect', '--exist-ok', '--save-crop']
    result = subprocess.run(command, capture_output=True, text=True)
    print('detect.py has finished running')
    print(result.stdout)
    print(result.stderr)

    # Add a delay to ensure the file is fully written
    time.sleep(5)

    # Gather detected frames
    detected_frames = []
    if os.path.exists(crops_dir):
        for file in os.listdir(crops_dir):
            if file.endswith(('.jpg', '.jpeg', '.png')):
                img_path = os.path.join(crops_dir, file)
                img = Image.open(img_path)
                detected_frames.append(img)

    return video_output, detected_frames

def create_collage(frames, cols=5):
    if len(frames) == 0:
        return None

    # Determine the size of the collage
    frame_width, frame_height = frames[0].size
    rows = (len(frames) + cols - 1) // cols

    collage_width = cols * frame_width
    collage_height = rows * frame_height

    # Create a blank collage
    collage = Image.new('RGB', (collage_width, collage_height))

    # Fill the collage with frames
    for idx, frame in enumerate(frames):
        row = idx // cols
        col = idx % cols
        collage.paste(frame, (col * frame_width, row * frame_height))

    return collage

@st.cache_data
def process_video_pytorchvideo(video_path):
    # Load the pre-trained SlowFast model
    model = slowfast_r101(pretrained="kinetics_600")
    model = model.eval()

    # Load the video using PyTorchVideo's EncodedVideo class
    video = EncodedVideo.from_path(video_path)

    # Select the time range to sample from the video
    start_time = 0
    end_time = min(video.duration, 15)  # Use the shorter of 15 seconds or video duration

    # Sample frames uniformly
    video_data = video.get_clip(start_sec=start_time, end_sec=end_time)
    frames = video_data["video"]

    # Define the transforms to apply to each frame
    transform = transforms.Compose([
        UniformTemporalSubsample(32),
        ShortSideScale(256),
        transforms.CenterCrop(224),
        transforms.Lambda(lambda x: x / 255.0),
    ])

    # Apply the initial transforms
    frames = transform(frames)

    # Normalize the RGB channels
    normalize = transforms.Normalize(mean=[0.45, 0.45, 0.45], std=[0.225, 0.225, 0.225])
    frames = frames.permute(1, 0, 2, 3)
    frames = normalize(frames)
    frames = frames.permute(1, 0, 2, 3)

    # Prepare the input format for SlowFast
    fast_pathway = frames
    slow_pathway = torch.index_select(
        frames,
        1,
        torch.linspace(0, frames.shape[1] - 1, frames.shape[1] // 4).long(),
    )

    # Combine pathways into a list and add a batch dimension
    inputs = [slow_pathway.unsqueeze(0), fast_pathway.unsqueeze(0)]

    # Run inference
    with torch.no_grad():
        preds = model(inputs)

    # Load Kinetics-400 class labels
    kinetics_labels = []
    with open('/content/label_map.txt', 'r') as f:
        kinetics_labels = [line.strip() for line in f if line.strip()]

    # Get top 5 predictions
    top5_pred = torch.topk(preds, k=5)
    top5_actions = []
    for i in range(5):
        if i < len(kinetics_labels):
            action = kinetics_labels[top5_pred.indices[0][i]]
            confidence = top5_pred.values[0][i].item()
            top5_actions.append((action, confidence))
        else:
            break

    return top5_actions

def process_video(uploaded_file):
    if uploaded_file is not None:
        video_path = f'/content/uploadedvideo{uploaded_file.name}'
        with open(video_path, "wb") as f:
            f.write(uploaded_file.getbuffer())

        # Display the original uploaded video
        st.video(uploaded_file)

        # YOLOv5 Inference
        st.subheader(f'YOLOv5 Cell Phone Detection for {uploaded_file.name}')
        yolov5_output, detected_frames = process_video_yolov5(video_path)

        if detected_frames:
            st.write(f'True Positive: {len(detected_frames)} frames detected with cell phones')
            collage_image = create_collage(detected_frames)
            if collage_image:
                st.image(collage_image, caption='Collage of detected frames')
        else:
            st.write('False Positive: No frames with cell phones detected')

        # Provide a download button for the processed video
        with open(yolov5_output, "rb") as file:
            video_bytes = file.read()
        st.write("You can download the processed video:")
        st.download_button(
            label="Download processed video",
            data=video_bytes,
            file_name=f"processedvideo{uploaded_file.name}",
            mime="video/mp4"
        )

        # SlowFast Inference
        st.subheader(f'PyTorchVideo Action Recognition for {uploaded_file.name}')
        slowfast_output = process_video_pytorchvideo(video_path)

        st.write('Top 5 predicted actions:')
        for i, (action, confidence) in enumerate(slowfast_output, 1):
            st.write(f"{i}. {action}: {confidence:.3f}")

# Streamlit app logic
st.title('YOLOv5 Cell Phone Detection and PyTorchVideo Action Recognition')

# Allow multiple file uploads
uploaded_files = st.file_uploader('Upload video files', type=['mp4', 'avi', 'mov'], accept_multiple_files=True)

if uploaded_files:
    for uploaded_file in uploaded_files:
        process_video(uploaded_file)

# Add Another Video functionality
if st.button("Add Another Video"):
    additional_file = st.file_uploader('Upload another video file', type=['mp4', 'avi', 'mov'], key='additional_upload')
    if additional_file:
        process_video(additional_file)

"""
with open("app.py", "w") as file:
    file.write(app_code)
print("app.py file has been written.")

# Start the Streamlit app in the background
import subprocess
streamlit_process = subprocess.Popen(['streamlit', 'run', 'app.py'])

# Start LocalTunnel
import time
import requests

time.sleep(5)
localtunnel_process = subprocess.Popen(['lt', '--port', '8501'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

# Capture the LocalTunnel URL
public_url = None
for line in iter(localtunnel_process.stdout.readline, b''):
    decoded_line = line.decode().strip()
    print(decoded_line)
    if 'your url is:' in decoded_line:
        public_url = decoded_line.split(' ')[-1]
        break

if public_url:
    print(f'Streamlit app running at: {public_url}')
    headers = {
        'User-Agent': 'Mozilla/5.0',
        'bypass-tunnel-reminder': '1'
    }
    response = requests.get(public_url, headers=headers)
    if response.status_code == 200:
        print(f'Access the Streamlit app at: {public_url}')
    else:
        print('Failed to bypass the password screen.')
else:
    print('Failed to create LocalTunnel.')

  Cloning https://github.com/facebookresearch/fvcore.git to /tmp/pip-req-build-9l73qzvf
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/fvcore.git /tmp/pip-req-build-9l73qzvf
  Resolved https://github.com/facebookresearch/fvcore.git to commit f3b07ea37daff5b774702708134f7b702e2dc83e
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/facebookresearch/fairscale.git to /tmp/pip-req-build-dja_bthq
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/fairscale.git /tmp/pip-req-build-dja_bthq
  Resolved https://github.com/facebookresearch/fairscale.git to commit 5f484b3545f27eddb19d970fbe1d361b9c5f2b07
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.7/132.7 kB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!wget -q -O - https://loca.lt/mytunnelpassword

34.125.25.245